# Wheel Strategy v4: CSP Assignment → Covered Call Module

This notebook extends the CSP backtest to handle the full wheel cycle:
1. Sell Cash-Secured Put (CSP)
2. If assigned (expires ITM): take stock delivery
3. Sell Covered Call (CC) against assigned shares
4. If called away (CC expires ITM): wheel complete

## Key Features
- **State Machine Architecture**: Explicit trade lifecycle states prevent logic spaghetti
- **Unified P&L Accounting**: CSP + CC + Stock P&L properly aggregated
- **v1 Scope**: One CC cycle per assignment (no rolling, no early assignment)

## Trade States
```
CSP_OPEN → CSP_CLOSED_PROFIT | CSP_CLOSED_STOP | CSP_ASSIGNED | CSP_CLOSED_WORTHLESS
CSP_ASSIGNED → CC_OPEN
CC_OPEN → CC_CLOSED_PROFIT | CC_ASSIGNED | CC_CLOSED_WORTHLESS
All terminal states → WHEEL_COMPLETE
```


## Imports

In [10]:
from pathlib import Path
from dotenv import dotenv_values, load_dotenv
import sys
import os
import pandas as pd
import databento as db
import pandas_market_calendars as mcal

sys.executable

env_path = Path("/Users/samuelminer/Projects/nissan_options/wheel_strategy/.env")

print("Parsed keys:", dotenv_values(env_path).keys())

load_dotenv()  # loads .env from current working directory

assert os.getenv("DATABENTO_API_KEY"), "DATABENTO_API_KEY still not found"
print("os.getenv:", bool(os.getenv("DATABENTO_API_KEY")))
client = db.Historical()

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)


Parsed keys: odict_keys(['DATABENTO_API_KEY', 'ANTHROPIC_API_KEY'])
os.getenv: True


## Configuration

All configurable parameters for the backtest. Modify this cell to change settings.

In [33]:
# =============================================================================
# UNIFIED CONFIGURATION
# =============================================================================

CONFIG = {
    # -------------------------------------------------------------------------
    # SYMBOL & TIMING
    # -------------------------------------------------------------------------
    'symbol': 'TSLA',                          # Underlying symbol to backtest
    'timezone': 'America/New_York',
    
    # Entry date/time for the single-day backtest
    'entry_date': '2023-06-06',                # Date to enter positions
    'entry_time': '15:45',                     # Time to capture option chain snapshot
    
    # Historical data lookback for technical indicators (e.g., Bollinger Bands)
    'lookback_days': 252 * 2,                  # ~2 years of daily data
    
    # -------------------------------------------------------------------------
    # OPTION SELECTION CRITERIA
    # -------------------------------------------------------------------------
    'option_type': 'P',                        # 'P' for puts (CSP), 'C' for calls
    'dte_min': 30,                             # Minimum days to expiration
    'dte_max': 45,                             # Maximum days to expiration
    'delta_min': 0.25,                         # Minimum absolute delta
    'delta_max': 0.35,                         # Maximum absolute delta
    
    # -------------------------------------------------------------------------
    # LIQUIDITY MODEL (regime-aware, penalty-based)
    # -------------------------------------------------------------------------
    # Hard rejection thresholds (truly untradeable)
    'min_bid_hard': 0.10,                      # Hard floor - reject penny options
    'hard_max_spread_pct': 0.20,               # Hard ceiling - reject extreme spreads
    
    # Base target spread (calm market conditions)
    'base_max_spread_pct': 0.08,               # Target max spread in normal conditions
    
    # IV regime adjustments (allow wider spreads in high-vol)
    'ivp_high_threshold': 0.70,                # IV percentile threshold for "high vol"
    'ivp_high_max_spread_pct': 0.12,           # Allowed spread when IV is high
    'ivp_extreme_threshold': 0.90,             # IV percentile threshold for "extreme vol"
    'ivp_extreme_max_spread_pct': 0.15,        # Allowed spread when IV is extreme
    
    # DTE adjustments (short-dated options have wider spreads)
    'short_dte_threshold': 7,                  # DTE below this gets extra allowance
    'short_dte_extra_spread_pct': 0.02,        # Extra spread allowance for short DTE
    
    # Penalty tiers (execution tax based on spread quality)
    # tight:    spread <= 0.6 * allowed → penalty = 1.0 (no extra slippage)
    # moderate: spread <= allowed       → penalty = 1.15 (15% wider effective spread)
    # wide:     spread <= hard_max      → penalty = 1.35 (35% wider effective spread)
    # ugly:     spread > hard_max       → REJECT (no trade)
    
    # -------------------------------------------------------------------------
    # EXIT STRATEGY
    # -------------------------------------------------------------------------
    'exit_pct': 0.50,                          # 0.50 = buy back at 50%, keep 50% profit
    'stop_loss_multiplier': 20.0,               # Exit if option price reaches Nx premium
    'max_hold_dte': None,                      # Exit at X DTE if no other trigger (None = disabled)
    
    # -------------------------------------------------------------------------
    # TRANSACTION COSTS (NEW - will be applied later)
    # -------------------------------------------------------------------------
    'commission_per_contract': 0.65,           # Per contract commission (round trip = 2x)
    'sec_fee_per_contract': 0.01,              # SEC/TAF fees per contract
    
    # -------------------------------------------------------------------------
    # EXECUTION / FILL ASSUMPTIONS (NEW - will be applied later)
    # -------------------------------------------------------------------------
    'fill_mode': 'mid',                        # 'mid' (current), 'bid' (realistic), 'pessimistic'
    'use_realistic_fills': False,              # When True: sell at bid, buy back at ask
    
    # -------------------------------------------------------------------------
    # PROBABILISTIC EXIT FILLS
    # -------------------------------------------------------------------------
    'execution_seed': 42,                      # Random seed for reproducible fills
    'use_probabilistic_exit_fills': True,      # Enable probabilistic fill model
    
    # Fill probability buckets by spread quality
    'pfill_tight': 0.90,                       # spread <= 5%
    'pfill_normal': 0.70,                      # spread <= 10%
    'pfill_wide': 0.40,                        # spread > 10%
    
    # Spread thresholds for buckets
    'tight_spread_pct': 0.05,
    'normal_spread_pct': 0.10,
    
    # Scaling and clamping
    'pfill_scale': 0.6,                        # Sensitivity multiplier (0.8, 1.0, 1.2)
    'pfill_min': 0.05,
    'pfill_max': 0.98,
    
    # Optional IVP penalty multipliers
    'pfill_ivp_high_mult': 0.85,
    'pfill_ivp_extreme_mult': 0.70,
    
    # -------------------------------------------------------------------------
    # CACHE
    # -------------------------------------------------------------------------
    'cache_dir': '../cache/',
}

# -------------------------------------------------------------------------
# DERIVED VALUES (computed from CONFIG)
# -------------------------------------------------------------------------
SYMBOL = CONFIG['symbol']
TZ = CONFIG['timezone']
CACHE_DIR = CONFIG['cache_dir']
os.makedirs(CACHE_DIR, exist_ok=True)

# Entry timestamp
ENTRY_DATE = pd.Timestamp(CONFIG['entry_date'], tz=TZ)
ENTRY_TIME = pd.Timestamp(f"{CONFIG['entry_date']} {CONFIG['entry_time']}", tz=TZ)

print("=" * 60)
print("BACKTEST CONFIGURATION")
print("=" * 60)
print(f"Symbol:          {SYMBOL}")
print(f"Entry Date:      {ENTRY_DATE.date()}")
print(f"Entry Time:      {CONFIG['entry_time']}")
print(f"Option Type:     {'Cash-Secured Put' if CONFIG['option_type'] == 'P' else 'Covered Call'}")
print(f"DTE Range:       {CONFIG['dte_min']} - {CONFIG['dte_max']} days")
print(f"Delta Range:     {CONFIG['delta_min']} - {CONFIG['delta_max']}")
print(f"Exit Target:     {CONFIG['exit_pct']*100:.0f}% of premium")
print(f"Stop Loss:       {CONFIG['stop_loss_multiplier']}x premium")
print(f"Fill Mode:       {CONFIG['fill_mode']}")
print(f"Realistic Fills: {CONFIG['use_realistic_fills']}")
print(f"Commission:      ${CONFIG['commission_per_contract']}/contract")
print("=" * 60)
print("\nNOTE: Transaction costs and realistic fills are NOT yet applied.")
print("      Run both notebooks to compare baseline vs realistic results.")

BACKTEST CONFIGURATION
Symbol:          TSLA
Entry Date:      2023-06-06
Entry Time:      15:45
Option Type:     Cash-Secured Put
DTE Range:       30 - 45 days
Delta Range:     0.25 - 0.35
Exit Target:     50% of premium
Stop Loss:       20.0x premium
Fill Mode:       mid
Realistic Fills: False
Commission:      $0.65/contract

NOTE: Transaction costs and realistic fills are NOT yet applied.
      Run both notebooks to compare baseline vs realistic results.


In [34]:
# =============================================================================
# COVERED CALL CONFIGURATION
# =============================================================================

CC_CONFIG = {
    # -------------------------------------------------------------------------
    # OPTION SELECTION CRITERIA
    # -------------------------------------------------------------------------
    'dte_min': 14,                             # Minimum days to expiration
    'dte_max': 30,                             # Maximum days to expiration
    'delta_min': 0.25,                         # Minimum absolute delta
    'delta_max': 0.35,                         # Maximum absolute delta
    'strike_min_pct_above_basis': 0.0,         # Allow ATM (0% above cost basis)
    'entry_time': '15:45',                     # Same snapshot time as CSP
    
    # -------------------------------------------------------------------------
    # BEHAVIORAL FLAGS (explicit intent documentation)
    # -------------------------------------------------------------------------
    'sell_call_only_if_price_above_basis': True,  # Require strike >= cost basis per share
    
    # -------------------------------------------------------------------------
    # TIE-BREAKING FOR CALL SELECTION
    # -------------------------------------------------------------------------
    # When multiple candidates match criteria, how to select:
    # Options: 'highest_premium', 'closest_delta', 'highest_strike'
    'tie_break_method': 'highest_premium',
}

print("=" * 60)
print("COVERED CALL CONFIGURATION")
print("=" * 60)
print(f"DTE Range:       {CC_CONFIG['dte_min']} - {CC_CONFIG['dte_max']} days")
print(f"Delta Range:     {CC_CONFIG['delta_min']} - {CC_CONFIG['delta_max']}")
print(f"Entry Time:      {CC_CONFIG['entry_time']}")
print(f"Strike >= Basis: {CC_CONFIG['sell_call_only_if_price_above_basis']}")
print(f"Tie-Breaking:    {CC_CONFIG['tie_break_method']}")
print("=" * 60)



COVERED CALL CONFIGURATION
DTE Range:       14 - 30 days
Delta Range:     0.25 - 0.35
Entry Time:      15:45
Strike >= Basis: True
Tie-Breaking:    highest_premium


In [35]:
# =============================================================================
# V5 SCHEDULER CONFIGURATION
# =============================================================================

SCHEDULER_CONFIG = {
    # -------------------------------------------------------------------------
    # DATE RANGE
    # -------------------------------------------------------------------------
    'start_date': '2023-06-06',                # Start date for backtest
    'end_date': '2023-09-13',                  # End date for backtest
    'trading_calendar': 'NYSE',                # Market calendar to use
    
    # -------------------------------------------------------------------------
    # SYMBOLS
    # -------------------------------------------------------------------------
    'symbols': ['TSLA'],                       # List of symbols to backtest
    
    # -------------------------------------------------------------------------
    # WHEEL INSTANCE CONTROL
    # -------------------------------------------------------------------------
    'allow_multiple_wheels_per_symbol': True,  # Allow overlapping wheels
    'max_wheels_per_symbol_per_day': None,     # None = no limit
    
    # -------------------------------------------------------------------------
    # REPRODUCIBILITY
    # -------------------------------------------------------------------------
    'scheduler_seed': 123,                     # Reserved for v6 ranking/capital allocation
    
    # -------------------------------------------------------------------------
    # LOGGING
    # -------------------------------------------------------------------------
    'log_level': 'INFO',                       # 'INFO' or 'QUIET'
}

print("=" * 60)
print("V5 SCHEDULER CONFIGURATION")
print("=" * 60)
print(f"Date Range:      {SCHEDULER_CONFIG['start_date']} to {SCHEDULER_CONFIG['end_date']}")
print(f"Calendar:        {SCHEDULER_CONFIG['trading_calendar']}")
print(f"Symbols:         {SCHEDULER_CONFIG['symbols']}")
print(f"Multi-Wheel:     {SCHEDULER_CONFIG['allow_multiple_wheels_per_symbol']}")
print(f"Log Level:       {SCHEDULER_CONFIG['log_level']}")
print("=" * 60)


V5 SCHEDULER CONFIGURATION
Date Range:      2023-06-06 to 2023-09-13
Calendar:        NYSE
Symbols:         ['TSLA']
Multi-Wheel:     True
Log Level:       INFO


In [36]:

# =============================================================================
# WHEEL STATE MACHINE
# =============================================================================
# Explicit state transitions prevent logic spaghetti and make logs interpretable.
# WHEEL_COMPLETE is the single canonical terminal state for all paths.

import uuid

# Valid state transitions
VALID_TRANSITIONS = {
    'CSP_OPEN': ['CSP_CLOSED_PROFIT', 'CSP_CLOSED_STOP', 'CSP_ASSIGNED', 'CSP_CLOSED_WORTHLESS'],
    'CSP_ASSIGNED': ['CC_OPEN', 'WHEEL_COMPLETE'],  # Can sell CC or mark incomplete
    'CC_OPEN': ['CC_CLOSED_PROFIT', 'CC_ASSIGNED', 'CC_CLOSED_WORTHLESS'],
    'CC_ASSIGNED': ['WHEEL_COMPLETE'],
    'CC_CLOSED_PROFIT': ['WHEEL_COMPLETE'],      # v1: no re-entry after CC profit
    'CC_CLOSED_WORTHLESS': ['WHEEL_COMPLETE'],   # v1: no re-entry after CC expires
    'CSP_CLOSED_PROFIT': ['WHEEL_COMPLETE'],
    'CSP_CLOSED_STOP': ['WHEEL_COMPLETE'],
    'CSP_CLOSED_WORTHLESS': ['WHEEL_COMPLETE'],
    'WHEEL_COMPLETE': [],  # Terminal state - no further transitions
}

# Event to state mapping
EVENT_TO_STATE = {
    # CSP phase events
    ('CSP_OPEN', 'profit_target'): 'CSP_CLOSED_PROFIT',
    ('CSP_OPEN', 'stop_loss'): 'CSP_CLOSED_STOP',
    ('CSP_OPEN', 'assigned'): 'CSP_ASSIGNED',
    ('CSP_OPEN', 'expired_worthless'): 'CSP_CLOSED_WORTHLESS',
    
    # Assignment to CC
    ('CSP_ASSIGNED', 'sell_call'): 'CC_OPEN',
    
    # CC phase events
    ('CC_OPEN', 'profit_target'): 'CC_CLOSED_PROFIT',
    ('CC_OPEN', 'called_away'): 'CC_ASSIGNED',
    ('CC_OPEN', 'expired_worthless'): 'CC_CLOSED_WORTHLESS',
    
    # Terminal transitions (all paths lead to WHEEL_COMPLETE)
    ('CC_ASSIGNED', 'complete'): 'WHEEL_COMPLETE',
    ('CC_CLOSED_PROFIT', 'complete'): 'WHEEL_COMPLETE',
    ('CC_CLOSED_WORTHLESS', 'complete'): 'WHEEL_COMPLETE',
    ('CSP_CLOSED_PROFIT', 'complete'): 'WHEEL_COMPLETE',
    ('CSP_CLOSED_STOP', 'complete'): 'WHEEL_COMPLETE',
    ('CSP_CLOSED_WORTHLESS', 'complete'): 'WHEEL_COMPLETE',
    ('CSP_ASSIGNED', 'complete'): 'WHEEL_COMPLETE',  # For incomplete wheels (no CC processed)
}


def advance_wheel_state(current_state, event):
    """
    Advance wheel state based on event. Enforces valid transitions.
    
    Args:
        current_state: Current state string (e.g., 'CSP_OPEN', 'CC_OPEN')
        event: Event triggering transition (e.g., 'profit_target', 'assigned', 'called_away')
    
    Returns:
        New state string
    
    Raises:
        ValueError if transition is invalid
    
    Example:
        >>> advance_wheel_state('CSP_OPEN', 'assigned')
        'CSP_ASSIGNED'
        >>> advance_wheel_state('CC_OPEN', 'called_away')
        'CC_ASSIGNED'
    """
    key = (current_state, event)
    
    if key not in EVENT_TO_STATE:
        valid_events = [e for (s, e) in EVENT_TO_STATE.keys() if s == current_state]
        raise ValueError(
            f"Invalid transition: state='{current_state}' + event='{event}'. "
            f"Valid events from {current_state}: {valid_events}"
        )
    
    new_state = EVENT_TO_STATE[key]
    
    # Double-check against VALID_TRANSITIONS (belt and suspenders)
    if new_state not in VALID_TRANSITIONS.get(current_state, []):
        raise ValueError(
            f"State '{new_state}' not reachable from '{current_state}'. "
            f"Valid transitions: {VALID_TRANSITIONS.get(current_state, [])}"
        )
    
    return new_state


def generate_wheel_id():
    """Generate a unique wheel ID for linking CSP + CC phases."""
    return str(uuid.uuid4())[:8]


def get_phase_from_state(state):
    """
    Extract phase from state string.
    
    Returns:
        'csp': For CSP states (CSP_OPEN, CSP_CLOSED_*, CSP_ASSIGNED)
        'cc': For CC states (CC_OPEN, CC_CLOSED_*, CC_ASSIGNED)
        'total': For WHEEL_COMPLETE (used in wheel summaries)
        'unknown': For unrecognized states
    """
    if state.startswith('CSP'):
        return 'csp'
    elif state.startswith('CC'):
        return 'cc'
    elif state == 'WHEEL_COMPLETE':
        return 'total'  # Consistent with wheel summary phase
    else:
        return 'unknown'


def is_terminal_state(state):
    """Check if state is a terminal state (no further transitions possible)."""
    return len(VALID_TRANSITIONS.get(state, [])) == 0 or state == 'WHEEL_COMPLETE'


# Test the state machine
print("=" * 60)
print("STATE MACHINE VALIDATION")
print("=" * 60)

# Test valid transitions
test_cases = [
    ('CSP_OPEN', 'profit_target', 'CSP_CLOSED_PROFIT'),
    ('CSP_OPEN', 'assigned', 'CSP_ASSIGNED'),
    ('CSP_ASSIGNED', 'sell_call', 'CC_OPEN'),
    ('CSP_ASSIGNED', 'complete', 'WHEEL_COMPLETE'),  # For incomplete wheels
    ('CC_OPEN', 'called_away', 'CC_ASSIGNED'),
    ('CC_ASSIGNED', 'complete', 'WHEEL_COMPLETE'),
]

for current, event, expected in test_cases:
    result = advance_wheel_state(current, event)
    status = "✓" if result == expected else "✗"
    print(f"  {status} {current} + '{event}' → {result}")

# Test invalid transition (should raise)
try:
    advance_wheel_state('CSP_OPEN', 'called_away')  # Invalid: called_away only valid for CC_OPEN
    print("  ✗ Should have raised ValueError for invalid transition")
except ValueError as e:
    print(f"  ✓ Correctly rejected invalid transition: CSP_OPEN + 'called_away'")

print("=" * 60)


STATE MACHINE VALIDATION
  ✓ CSP_OPEN + 'profit_target' → CSP_CLOSED_PROFIT
  ✓ CSP_OPEN + 'assigned' → CSP_ASSIGNED
  ✓ CSP_ASSIGNED + 'sell_call' → CC_OPEN
  ✓ CSP_ASSIGNED + 'complete' → WHEEL_COMPLETE
  ✓ CC_OPEN + 'called_away' → CC_ASSIGNED
  ✓ CC_ASSIGNED + 'complete' → WHEEL_COMPLETE
  ✓ Correctly rejected invalid transition: CSP_OPEN + 'called_away'


In [37]:
# =============================================================================
# HELPER FUNCTIONS FOR REALISTIC EXECUTION
# =============================================================================

def get_entry_price(row, fill_mode='realistic', penalty=1.0):
    """
    Calculate entry price when SELLING a put (we receive premium).
    Higher price = better for us.
    
    Slippage is calculated as a percentage of the bid-ask spread from mid.
    Penalty multiplier widens the effective spread for illiquid options.
    
    | Scenario    | Formula                              | Interpretation              |
    |-------------|--------------------------------------|-----------------------------|
    | pessimistic | mid - 75% of (spread * penalty)      | Forced/stressed execution   |
    | realistic   | mid - 30% of (spread * penalty)      | Normal retail execution     |
    | optimistic  | mid                                  | Patient, favorable fills    |
    
    Args:
        row: DataFrame row with bid_px_00, ask_px_00
        fill_mode: 'optimistic', 'realistic', or 'pessimistic'
        penalty: liquidity penalty multiplier (1.0 = no extra slippage)
    """
    bid = row['bid_px_00']
    ask = row['ask_px_00']
    mid = (bid + ask) / 2
    spread = ask - bid
    
    # Apply liquidity penalty to effective spread
    effective_spread = spread * penalty
    
    if fill_mode == 'optimistic':
        return mid                              # Best case - get mid (no penalty applied)
    elif fill_mode == 'pessimistic':
        fill = mid - (0.75 * effective_spread)  # Worst case - 75% toward bid
    else:  # realistic
        fill = mid - (0.30 * effective_spread)  # Normal - 30% toward bid
    
    # Clamp to [bid, ask] to stay realistic
    return max(bid, min(ask, fill))


def get_exit_price(daily_row, fill_mode=CONFIG['fill_mode'], target_price=None, penalty=1.0): # IS THIS RIGHT TO SET AT PENALTY = 1.0? ##
    """
    Calculate exit price when BUYING BACK a put (we pay to close).
    Lower price = better for us.
    
    For daily OHLCV data, we estimate spread behavior from the day's range.
    Penalty multiplier widens the effective range for illiquid options.
    
    | Scenario    | Formula                              | Interpretation              |
    |-------------|--------------------------------------|-----------------------------|
    | pessimistic | close + 75% of (range * penalty)     | Forced/stressed execution   |
    | realistic   | close + 30% of (range * penalty)     | Normal retail execution     |
    | optimistic  | close - 25% of (range * penalty)     | Patient, favorable fills    |
    
    Args:
        daily_row: DataFrame row with close, high, low
        fill_mode: 'optimistic', 'realistic', or 'pessimistic'
        target_price: Optional target price (not currently used but reserved)
        penalty: liquidity penalty multiplier (1.0 = no extra slippage)
    """
    close = daily_row['close']
    high = daily_row['high']
    low = daily_row['low']
    day_range = high - low  # Proxy for intraday spread/volatility
    
    # Apply liquidity penalty to effective range
    effective_range = day_range * penalty
    
    if fill_mode == 'optimistic':
        # Patient buyer - gets below close (toward low)
        fill = close - (0.25 * effective_range)
        return max(low, fill)
    elif fill_mode == 'pessimistic':
        # Forced buyer - pays above close (toward high)
        fill = close + (0.75 * effective_range)
        return min(high, fill)
    else:  # realistic
        # Normal execution - slight slippage above close
        fill = close + (0.30 * effective_range)
        return min(high, fill)


def get_transaction_costs(config, is_round_trip=True):
    """
    Calculate total transaction costs per contract.
    
    Args:
        config: CONFIG dict with commission and fee rates
        is_round_trip: True if both entry and exit, False if entry only (e.g., expired worthless)
    
    Returns:
        Total fees in dollars per contract
    """
    per_leg = config['commission_per_contract'] + config['sec_fee_per_contract']
    return per_leg * 2 if is_round_trip else per_leg


def compute_allowed_spread(row, config):
    """
    Compute the allowed spread percentage for a single option based on regime.
    
    Regime factors:
    - IV percentile (high vol → allow wider spreads)
    - DTE (short-dated → allow wider spreads)
    
    Returns: allowed_spread_pct for this option
    """
    base = config['base_max_spread_pct']
    
    # IV regime adjustment
    ivp = row.get('ivp', 0.5)  # Default to median if not computed
    if ivp >= config['ivp_extreme_threshold']:
        base = config['ivp_extreme_max_spread_pct']
    elif ivp >= config['ivp_high_threshold']:
        base = config['ivp_high_max_spread_pct']
    
    # DTE adjustment
    dte = row.get('dte', 30)
    if dte <= config['short_dte_threshold']:
        base += config['short_dte_extra_spread_pct']
    
    return base


def compute_liquidity_penalty(spread_pct, allowed_spread_pct, hard_max_spread_pct):
    """
    Compute liquidity penalty multiplier based on spread quality.
    
    Tiers:
    - tight:    spread <= 0.6 * allowed → penalty = 1.0 (no extra slippage)
    - moderate: spread <= allowed       → penalty = 1.15
    - wide:     spread <= hard_max      → penalty = 1.35
    - ugly:     spread > hard_max       → None (reject)
    
    Returns: (tier_name, penalty_multiplier) or (None, None) if rejected
    """
    if spread_pct > hard_max_spread_pct:
        return 'reject', None
    
    tight_threshold = 0.6 * allowed_spread_pct
    
    if spread_pct <= tight_threshold:
        return 'tight', 1.0
    elif spread_pct <= allowed_spread_pct:
        return 'moderate', 1.15
    else:  # spread_pct <= hard_max_spread_pct
        return 'wide', 1.35


def apply_liquidity_model(df, config):
    """
    Apply regime-aware liquidity model with penalty tiers.
    
    Instead of binary reject, this:
    1. Computes IV percentile (ivp) for regime detection
    2. Computes allowed_spread_pct per option (regime-aware)
    3. Assigns liquidity_tier and liquidity_penalty
    4. Only hard-rejects truly ugly spreads
    
    Args:
        df: DataFrame with option quotes (needs bid_px_00, ask_px_00, spread_pct, iv, dte)
        config: CONFIG dict with liquidity model settings
    
    Returns:
        DataFrame with liquidity columns added, ugly spreads removed
    """
    if len(df) == 0:
        return df
    
    df = df.copy()
    original_count = len(df)
    
    # Ensure required columns exist
    if 'spread_pct' not in df.columns:
        df['spread'] = df['ask_px_00'] - df['bid_px_00']
        df['spread_pct'] = df['spread'] / df['mid']
    
    # Step 1: Compute IV percentile (cross-sectional within this snapshot)
    if 'iv' in df.columns:
        df['ivp'] = df['iv'].rank(pct=True)
    else:
        df['ivp'] = 0.5  # Default to median if IV not available
    
    # Step 2: Compute allowed spread per option
    df['allowed_spread_pct'] = df.apply(
        lambda row: compute_allowed_spread(row, config), axis=1
    )
    
    # Step 3: Compute liquidity tier and penalty
    def get_tier_and_penalty(row):
        return compute_liquidity_penalty(
            row['spread_pct'], 
            row['allowed_spread_pct'],
            config['hard_max_spread_pct']
        )
    
    tiers_penalties = df.apply(get_tier_and_penalty, axis=1)
    df['liquidity_tier'] = tiers_penalties.apply(lambda x: x[0])
    df['liquidity_penalty'] = tiers_penalties.apply(lambda x: x[1])
    
    # Step 4: Hard reject only truly ugly spreads and penny options
    df = df[
        (df['liquidity_tier'] != 'reject') &
        (df['bid_px_00'] >= config['min_bid_hard'])
    ].copy()
    
    rejected = original_count - len(df)
    
    # Print diagnostics
    print(f"\n  Liquidity Model Applied:")
    print(f"    Original: {original_count} options")
    print(f"    Hard rejected: {rejected} ({rejected/original_count*100:.1f}%)")
    print(f"    Remaining: {len(df)} options")
    
    if len(df) > 0:
        tier_counts = df['liquidity_tier'].value_counts()
        print(f"    Tier breakdown: {dict(tier_counts)}")
        print(f"    Avg spread: {df['spread_pct'].mean()*100:.1f}%, Avg allowed: {df['allowed_spread_pct'].mean()*100:.1f}%")
        print(f"    Avg penalty: {df['liquidity_penalty'].mean():.2f}x")
    return df


def calculate_pnl(premium_received, exit_price_paid, fees, cost_basis):
    """
    Calculate P&L metrics for a trade.
    
    Args:
        premium_received: Premium collected when selling (contract value)
        exit_price_paid: Price paid to close position (contract value), 0 if expired worthless
        fees: Total transaction costs
        cost_basis: Capital at risk (strike * 100 for CSP)
    
    Returns:
        dict with pnl, pnl_pct, roc
    """
    pnl = premium_received - exit_price_paid - fees
    pnl_pct = (pnl / premium_received) * 100 if premium_received > 0 else 0
    roc = (pnl / cost_basis) * 100 if cost_basis > 0 else 0
    
    return {
        'pnl': pnl,
        'pnl_pct': pnl_pct,
        'roc': roc,
        'fees': fees
    }


def compute_p_fill_profit(row, config):
    """
    Compute probability of fill for profit target exit based on entry liquidity.
    
    Uses entry-time spread and IVP to determine fill probability:
    - tight spread (<=5%): high fill probability
    - normal spread (<=10%): moderate fill probability
    - wide spread (>10%): low fill probability
    
    Applies IVP penalty multipliers for high-volatility regimes.
    
    Args:
        row: DataFrame row with spread_pct_entry, ivp_entry
        config: CONFIG dict with fill probability settings
    
    Returns:
        float: Fill probability in [pfill_min, pfill_max]
    """
    spread_pct = row.get('spread_pct_entry', 0.05)
    
    # Bucket by spread quality
    if spread_pct <= config['tight_spread_pct']:
        p_fill = config['pfill_tight']
    elif spread_pct <= config['normal_spread_pct']:
        p_fill = config['pfill_normal']
    else:
        p_fill = config['pfill_wide']
    
    # Apply scale multiplier
    p_fill *= config['pfill_scale']
    
    # Apply IVP penalty (always use ivp_entry, not generic ivp)
    ivp = row.get('ivp_entry', 0.5)
    if ivp >= config['ivp_extreme_threshold']:
        p_fill *= config.get('pfill_ivp_extreme_mult', 1.0)
    elif ivp >= config['ivp_high_threshold']:
        p_fill *= config.get('pfill_ivp_high_mult', 1.0)
    
    # Clamp to valid range
    return max(config['pfill_min'], min(config['pfill_max'], p_fill))


def try_probabilistic_fill(p_fill, rng):
    """
    Simulate probabilistic fill by drawing uniform random number.
    
    Args:
        p_fill: Fill probability (0.0 to 1.0)
        rng: numpy random number generator
    
    Returns:
        tuple: (filled: bool, u: float) where u is the random draw
    """
    u = rng.uniform(0, 1)
    filled = (u <= p_fill)
    return filled, u


# Print summary of fill assumptions
print("=" * 60)
print("FILL ASSUMPTIONS BY SCENARIO")
print("=" * 60)
print(f"{'Scenario':<12} {'Entry (Sell)':<25} {'Exit (Buy Back)':<25}")
print("-" * 60)
print(f"{'Pessimistic':<12} {'Mid - 75% of spread':<25} {'Close + 75% of range':<25}")
print(f"{'Realistic':<12} {'Mid - 30% of spread':<25} {'Close + 30% of range':<25}")
print(f"{'Optimistic':<12} {'Mid (no slippage)':<25} {'Close - 25% of range':<25}")
print("=" * 60)
print(f"\nTransaction costs: ${CONFIG['commission_per_contract'] + CONFIG['sec_fee_per_contract']:.2f}/leg")
print(f"\nLiquidity Model (regime-aware):")
print(f"  Hard reject: bid < ${CONFIG['min_bid_hard']} or spread > {CONFIG['hard_max_spread_pct']*100:.0f}%")
print(f"  Base target spread: {CONFIG['base_max_spread_pct']*100:.0f}%")
print(f"  High IV ({CONFIG['ivp_high_threshold']*100:.0f}%ile): allow {CONFIG['ivp_high_max_spread_pct']*100:.0f}%")
print(f"  Extreme IV ({CONFIG['ivp_extreme_threshold']*100:.0f}%ile): allow {CONFIG['ivp_extreme_max_spread_pct']*100:.0f}%")
print(f"  Short DTE (≤{CONFIG['short_dte_threshold']}d): +{CONFIG['short_dte_extra_spread_pct']*100:.0f}% allowed")


FILL ASSUMPTIONS BY SCENARIO
Scenario     Entry (Sell)              Exit (Buy Back)          
------------------------------------------------------------
Pessimistic  Mid - 75% of spread       Close + 75% of range     
Realistic    Mid - 30% of spread       Close + 30% of range     
Optimistic   Mid (no slippage)         Close - 25% of range     

Transaction costs: $0.66/leg

Liquidity Model (regime-aware):
  Hard reject: bid < $0.1 or spread > 20%
  Base target spread: 8%
  High IV (70%ile): allow 12%
  Extreme IV (90%ile): allow 15%
  Short DTE (≤7d): +2% allowed


In [38]:
# =============================================================================
# V5 TRADING CALENDAR UTILITIES
# =============================================================================

def get_trading_days(start_date, end_date, calendar='NYSE'):
    """
    Get list of trading days between start and end dates.
    
    Args:
        start_date: Start date (string or Timestamp)
        end_date: End date (string or Timestamp)
        calendar: Market calendar name (default 'NYSE')
    
    Returns:
        DatetimeIndex of trading days
    
    Note:
        schedule.index is timezone-naive; we treat it as a date (NY trading day).
        The entry_time timezone logic is applied separately when building 
        config_day['entry_date'].
    """
    cal = mcal.get_calendar(calendar)
    schedule = cal.schedule(start_date=start_date, end_date=end_date)
    return schedule.index

# Test the utility
test_days = get_trading_days('2023-01-01', '2023-01-10')
print("=" * 60)
print("TRADING CALENDAR UTILITIES LOADED")
print("=" * 60)
print(f"Sample trading days (Jan 1-10, 2023):")
for day in test_days[:5]:
    print(f"  {day.date()} ({day.day_name()})")
print("=" * 60)


TRADING CALENDAR UTILITIES LOADED
Sample trading days (Jan 1-10, 2023):
  2023-01-03 (Tuesday)
  2023-01-04 (Wednesday)
  2023-01-05 (Thursday)
  2023-01-06 (Friday)
  2023-01-09 (Monday)


In [39]:
# =============================================================================
# V5 ENTRY CANDIDATE WRAPPER
# =============================================================================
# Encapsulates option chain fetching and filtering for a single symbol/date

import numpy as np
from py_vollib.black_scholes.implied_volatility import implied_volatility
from py_vollib.black_scholes.greeks.analytical import delta as calc_delta

def get_entry_candidates(symbol, trade_date, config, client):
    """
    Get qualifying CSP entry candidates for a symbol on a given date.
    
    This function encapsulates:
    1. Fetch option chain snapshot at entry_time
    2. Parse option symbols and calculate DTE
    3. Compute mid price, spread, IV, and delta
    4. Apply delta/DTE/liquidity filters
    5. Sort candidates deterministically for reproducibility
    
    Args:
        symbol: Underlying symbol (e.g., 'TSLA')
        trade_date: Trading date (pandas Timestamp)
        config: CONFIG dict with entry parameters
        client: Databento client
    
    Returns:
        DataFrame of qualifying CSP candidates (empty if none qualify)
    """
    cache_dir = config.get('cache_dir', '../cache/')
    tz = config.get('timezone', 'America/New_York')
    entry_time = config.get('entry_time', '15:45')
    
    # Build entry timestamp
    date_str = trade_date.strftime('%Y-%m-%d')
    entry_ts = pd.Timestamp(f"{date_str} {entry_time}", tz=tz)
    
    # Generate cache filename for options data
    date_cache_str = trade_date.strftime('%Y%m%d')
    time_cache_str = entry_ts.strftime('%H%M')
    cache_file = os.path.join(cache_dir, f"options_{symbol}_{date_cache_str}_{time_cache_str}.parquet")
    
    # Check cache first
    if os.path.exists(cache_file):
        df_opts = pd.read_parquet(cache_file)
    else:
        # Fetch from API
        try:
            start = entry_ts
            end = start + pd.Timedelta(minutes=1)
            
            data = client.timeseries.get_range(
                dataset='OPRA.PILLAR',
                schema='cmbp-1',
                symbols=f"{symbol}.OPT",
                stype_in='parent',
                start=start,
                end=end,
            )
            
            df_opts = data.to_df(tz=tz).sort_values("ts_event")
            df_opts.to_parquet(cache_file)
        except Exception as e:
            return pd.DataFrame()
    
    if len(df_opts) == 0:
        return pd.DataFrame()
    
    # Parse option symbols
    sym = df_opts["symbol"]
    root_and_code = sym.str.split(expand=True)
    df_opts["root"] = root_and_code[0]
    code = root_and_code[1]
    df_opts["expiration"] = pd.to_datetime(code.str[:6], format="%y%m%d")
    df_opts["call_put"] = code.str[6]
    strike_int = code.str[7:].astype("int32")
    df_opts["strike"] = strike_int / 1000.0
    expiration_tz = df_opts["expiration"].dt.tz_localize(df_opts["ts_event"].dt.tz)
    df_opts["dte"] = (expiration_tz - df_opts["ts_event"].dt.normalize()).dt.days
    
    # Filter by DTE and option type
    df_opts = df_opts[
        (df_opts['dte'] >= config['dte_min']) & 
        (df_opts['dte'] <= config['dte_max']) & 
        (df_opts['call_put'] == config['option_type'])
    ].copy()
    
    if len(df_opts) == 0:
        return pd.DataFrame()
    
    # Get underlying price at entry time
    equity_cache = os.path.join(cache_dir, f"equity_minute_{symbol}_{date_cache_str}_{time_cache_str}.parquet")
    if os.path.exists(equity_cache):
        equity_df = pd.read_parquet(equity_cache)
        underlying_price = equity_df['close'].iloc[-1] if len(equity_df) > 0 else None
    else:
        # Fetch from API
        try:
            equity_data = client.timeseries.get_range(
                dataset='XNAS.ITCH',
                symbols=[symbol],
                schema='ohlcv-1m',
                start=entry_ts,
                end=entry_ts + pd.Timedelta(minutes=1),
                stype_in='raw_symbol'
            )
            equity_df = equity_data.to_df()
            if len(equity_df) > 0:
                equity_df.to_parquet(equity_cache)
                underlying_price = equity_df['close'].iloc[-1]
            else:
                underlying_price = None
        except:
            underlying_price = None
    
    if underlying_price is None:
        return pd.DataFrame()
    
    # Filter to valid quotes only
    quotes = df_opts[df_opts["bid_px_00"].notna() & df_opts["ask_px_00"].notna()].copy()
    if len(quotes) == 0:
        return pd.DataFrame()
    
    # Compute mid price
    quotes["mid"] = (quotes["bid_px_00"] + quotes["ask_px_00"]) / 2
    quotes["spread"] = quotes["ask_px_00"] - quotes["bid_px_00"]
    quotes["spread_pct"] = quotes["spread"] / quotes["mid"]
    
    # Get last quote per contract
    chain_snapshot = (
        quotes
        .sort_values("ts_event")
        .groupby(["symbol", "expiration", "strike", "call_put"])
        .tail(1)
        .copy()
    )
    chain_snapshot["underlying_last"] = underlying_price
    
    # Compute IV and delta
    r = 0.04
    
    def compute_iv(row):
        price = row["mid"]
        S = row["underlying_last"]
        K = row["strike"]
        t = row["dte"] / 365.0
        flag = "p" if row["call_put"] == "P" else "c"
        
        if not (np.isfinite(price) and np.isfinite(S) and np.isfinite(K) and t > 0):
            return np.nan
        if price <= 0 or S <= 0 or K <= 0:
            return np.nan
        
        try:
            return implied_volatility(price, S, K, t, r, flag)
        except Exception:
            return np.nan
    
    def compute_delta(row):
        sigma = row["iv"]
        if not np.isfinite(sigma):
            return np.nan
        
        S = row["underlying_last"]
        K = row["strike"]
        t = row["dte"] / 365.0
        flag = "p" if row["call_put"] == "P" else "c"
        
        return abs(calc_delta(flag, S, K, t, r, sigma))
    
    chain_snapshot["iv"] = chain_snapshot.apply(compute_iv, axis=1)
    chain_snapshot["delta"] = chain_snapshot.apply(compute_delta, axis=1)
    chain_snapshot['date'] = chain_snapshot['ts_event'].dt.date
    
    # Filter by delta
    candidates = chain_snapshot[
        (chain_snapshot["call_put"] == config['option_type'])
        & chain_snapshot["dte"].between(config['dte_min'], config['dte_max'])
        & chain_snapshot["delta"].abs().between(config['delta_min'], config['delta_max'])
    ].copy()
    
    if len(candidates) == 0:
        return pd.DataFrame()
    
    # Apply liquidity model (silently)
    candidates = apply_liquidity_model_silent(candidates, config)
    
    if len(candidates) == 0:
        return pd.DataFrame()
    
    # Calculate entry price with liquidity penalty
    candidates['entry_price'] = candidates.apply(
        lambda row: get_entry_price(row, config['fill_mode'], row.get('liquidity_penalty', 1.0)), 
        axis=1
    )
    
    # Set up backtest fields
    candidates['per_share_premium'] = candidates['entry_price']
    candidates['premium'] = candidates['per_share_premium'] * 100
    candidates['cost_basis'] = candidates['strike'] * 100
    candidates['exit_pct'] = config['exit_pct']
    candidates['exit_price_per_share'] = candidates['per_share_premium'] * candidates['exit_pct']
    candidates['spread_pct_entry'] = candidates['spread_pct']
    candidates['ivp_entry'] = candidates['ivp']
    
    # Deterministic ordering for reproducibility
    candidates = candidates.sort_values(
        ['expiration', 'strike', 'symbol']
    ).reset_index(drop=True)
    
    return candidates


def apply_liquidity_model_silent(df, config):
    """
    Apply liquidity model without printing diagnostics.
    Same logic as apply_liquidity_model but silent.
    """
    if len(df) == 0:
        return df
    
    df = df.copy()
    
    if 'spread_pct' not in df.columns:
        df['spread'] = df['ask_px_00'] - df['bid_px_00']
        df['spread_pct'] = df['spread'] / df['mid']
    
    if 'iv' in df.columns:
        df['ivp'] = df['iv'].rank(pct=True)
    else:
        df['ivp'] = 0.5
    
    df['allowed_spread_pct'] = df.apply(
        lambda row: compute_allowed_spread(row, config), axis=1
    )
    
    def get_tier_and_penalty(row):
        return compute_liquidity_penalty(
            row['spread_pct'], 
            row['allowed_spread_pct'],
            config['hard_max_spread_pct']
        )
    
    tiers_penalties = df.apply(get_tier_and_penalty, axis=1)
    df['liquidity_tier'] = tiers_penalties.apply(lambda x: x[0])
    df['liquidity_penalty'] = tiers_penalties.apply(lambda x: x[1])
    
    df = df[
        (df['liquidity_tier'] != 'reject') &
        (df['bid_px_00'] >= config['min_bid_hard'])
    ].copy()
    
    return df


print("=" * 60)
print("V5 ENTRY CANDIDATE WRAPPER LOADED")
print("=" * 60)
print("  - get_entry_candidates(symbol, trade_date, config, client)")
print("  - apply_liquidity_model_silent(df, config)")
print("=" * 60)


V5 ENTRY CANDIDATE WRAPPER LOADED
  - get_entry_candidates(symbol, trade_date, config, client)
  - apply_liquidity_model_silent(df, config)


In [ ]:
def fetch_underlying_price_at_expiration(underlying_symbol, expiration_date, client, config):
    """
    Fetch underlying stock price at expiration date.
    
    Used to determine if a covered call is ITM (called away) or OTM (expires worthless).
    
    Args:
        underlying_symbol: Underlying symbol (e.g., 'TSLA')
        expiration_date: Expiration date (pandas Timestamp)
        client: Databento client
        config: Configuration dict with cache_dir and timezone
    
    Returns:
        float: Close price at expiration, or None if unavailable
    """
    cache_dir = config.get('cache_dir', '../cache/')
    tz = config.get('timezone', 'America/New_York')
    
    # Normalize expiration date
    if hasattr(expiration_date, 'tz') and expiration_date.tz:
        expiration_date = expiration_date.tz_localize(None)
    expiration_date = pd.Timestamp(expiration_date).normalize()
    
    # Try to fetch daily equity data for expiration date
    try:
        # Generate cache filename
        date_str = expiration_date.strftime('%Y%m%d')
        cache_file = os.path.join(cache_dir, f"equity_daily_{underlying_symbol}_{date_str}.parquet")
        
        # Check cache first
        if os.path.exists(cache_file):
            equity_df = pd.read_parquet(cache_file)
            if len(equity_df) > 0:
                # Return the close price
                return float(equity_df['close'].iloc[-1])
        
        # Cache miss - fetch from API
        start = pd.Timestamp(expiration_date, tz=tz)
        end = start + pd.Timedelta(days=1)
        
        equity_data = client.timeseries.get_range(
            dataset='EQUS.MINI',  # Consolidated US equities
            symbols=[underlying_symbol],
            schema='ohlcv-1d',
            start=start,
            end=end,
            stype_in='raw_symbol'
        )
        
        equity_df = equity_data.to_df(tz=tz)
        
        if len(equity_df) > 0:
            # Save to cache
            equity_df.to_parquet(cache_file)
            return float(equity_df['close'].iloc[-1])
        
    except Exception as e:
        print(f"    Warning: Could not fetch underlying price at expiration: {e}")
        return None
    
    return None
    
def fetch_daily_prices_for_option(symbol, entry_date, expiration_date, client, config):
    """
    Fetch daily OHLC prices for an option from entry date to expiration.

    Args:
        symbol: Option symbol
        entry_date: Entry date (normalized)
        expiration_date: Expiration date (normalized)
        client: Databento client
        config: Configuration dict

    Returns:
        DataFrame with daily OHLC data
    """
    # Generate cache filename for daily option prices
    entry_str = entry_date.strftime('%Y%m%d')
    exp_str = expiration_date.strftime('%Y%m%d')
    cache_file = os.path.join(CACHE_DIR, f"option_daily_{symbol}_{entry_str}_{exp_str}.parquet")

    # Check cache first
    if os.path.exists(cache_file):
        print(f"    [CACHE HIT] Loading daily prices for {symbol}")
        return pd.read_parquet(cache_file)

    # Cache miss - fetch from API
    print(f"    [API] Fetching daily prices for {symbol} from {entry_date.date()} to {expiration_date.date()}")

    start_daily = entry_date + pd.Timedelta(days=1)  # Day after entry
    end_daily = expiration_date + pd.Timedelta(days=1)  # Include expiration day

    daily_data = client.timeseries.get_range(
        dataset='OPRA.PILLAR',
        schema='ohlcv-1d',
        symbols=symbol,
        stype_in='raw_symbol',
        start=start_daily,
        end=end_daily,
    )

    df_daily = daily_data.to_df(tz=config['timezone'])

    # Save to cache
    df_daily.to_parquet(cache_file)
    print(f"    [CACHE SAVE] Saved {len(df_daily)} days to cache")

    return df_daily


def check_profit_target_hit(df_daily, exit_price_per_share, entry_date):
    """
    Check if the exit price target was hit in the daily price data.

    Args:
        df_daily: DataFrame with daily OHLC data (prices are per-share)
        exit_price_per_share: Target price per share to exit at
        entry_date: Entry date to skip (we can't exit same day we entered)

    Returns:
        tuple: (hit_date, daily_row) if hit, (None, None) if not hit
    """
    for check_date, daily_row in df_daily.iterrows():
        # Skip the entry date - we can't exit on the same day we entered
        check_date_normalized = check_date.tz_localize(None) if hasattr(check_date, 'tz_localize') and check_date.tz else check_date
        if check_date_normalized.date() <= entry_date.date():
            continue
            
        daily_low = daily_row['low']
        daily_high = daily_row['high']

        # Check if our exit target (per-share) is within the daily range
        if daily_low <= exit_price_per_share <= daily_high:
            return check_date, daily_row

    return None, None


def create_exit_record(symbol, entry_date, expiration_date, premium, exit_pct,
                       exit_price, exit_reason, check_date, daily_row, cost_basis,
                       wheel_id=None, initial_capital=None,  
                       touch_profit_target=None, p_fill_profit_target=None, u_fill_profit_target=None,
                       filled_profit_target=None, spread_pct_entry=None, ivp_entry=None,
                       touch_count=None):
    """
    Create an exit record dictionary.

    Args:
        symbol: Option symbol
        entry_date: Entry date
        expiration_date: Expiration date
        premium: Premium received
        exit_pct: Exit percentage (e.g., 0.25 = exit when decays 25%)
        exit_price: Actual exit price
        exit_reason: Reason for exit
        check_date: Date of exit
        daily_row: Daily price data row
        cost_basis: Cost basis (strike * 100)
        wheel_id: Unique identifier for this wheel instance  # ADD THIS
        initial_capital: Initial capital at risk (strike * 100 for CSP)  # ADD THIS
        touch_profit_target: Did price ever touch limit? (bool)
        p_fill_profit_target: Computed fill probability (float)
        u_fill_profit_target: Random draw value (float)
        filled_profit_target: Actually filled? (bool)
        spread_pct_entry: Entry-time spread % (float)
        ivp_entry: Entry-time IV percentile (float)
        touch_count: Number of distinct days with low <= L before fill (int)

    Returns:
        dict: Exit record
    """

        # Map exit_reason to state
    exit_reason_to_state = {
        'profit_target': 'CSP_CLOSED_PROFIT',
        'stop_loss': 'CSP_CLOSED_STOP',
        'expired_worthless': 'CSP_CLOSED_WORTHLESS',
        'assigned': 'CSP_ASSIGNED',
    }
    state = exit_reason_to_state.get(exit_reason, 'CSP_OPEN')

    return {
        'wheel_id': wheel_id,  # ADD THIS LINE
        'initial_capital': initial_capital,  # ADD THIS LINE
        'state': state,  # ADD THIS LINE - map exit_reason to state
        'symbol': symbol,
        'entry_date': entry_date,
        'exit_date': check_date.tz_localize(None) if hasattr(check_date, 'tz_localize') and check_date.tz else check_date,
        'expiration': expiration_date,
        'cost_basis': cost_basis,
        'premium': premium,
        'exit_pct': exit_pct,
        'exit_price': exit_price,
        'exit_reason': exit_reason,
        'days_held': (check_date.tz_localize(None) - entry_date).days if check_date else None,
        'daily_low': daily_row['low'] if daily_row is not None else None,
        'daily_high': daily_row['high'] if daily_row is not None else None,
        'touch_profit_target': touch_profit_target,
        'p_fill_profit_target': p_fill_profit_target,
        'u_fill_profit_target': u_fill_profit_target,
        'filled_profit_target': filled_profit_target,
        'spread_pct_entry': spread_pct_entry,
        'ivp_entry': ivp_entry,
        'touch_count': touch_count,
        'fill_model': 'ohlc_touch_prob_v1',
    }


def calculate_pnl_metrics(exits_df, config):
    """
    Calculate P&L metrics for exit results.

    Args:
        exits_df: DataFrame with exit records
        config: Configuration dict with fee settings

    Returns:
        DataFrame with P&L metrics added
    """
    if len(exits_df) > 0:
        exits_df = exits_df.copy()
        
        # Calculate transaction costs based on exit reason
        # Expired worthless = entry fee only (no buyback needed)
        # All other exits = round-trip fees
        exits_df['fees'] = exits_df['exit_reason'].apply(
            lambda reason: get_transaction_costs(config, is_round_trip=(reason != 'expired_worthless'))
        )
        
        # P&L after fees
        exits_df['exit_pnl'] = exits_df['premium'] - exits_df['exit_price'] - exits_df['fees']
        exits_df['exit_pnl_pct'] = (exits_df['exit_pnl'] / exits_df['premium']) * 100
        exits_df['roc'] = (exits_df['exit_pnl'] / exits_df['cost_basis']) * 100
        
        # Summary stats
        total_fees = exits_df['fees'].sum()
        print(f"\n  Transaction costs: ${total_fees:.2f} total ({len(exits_df)} trades)")

    return exits_df


def backtest_exit_strategy(backtest_candidates, client, config):
    """
    Backtest exit strategy for wheel options with probabilistic fills.

    Exit conditions:
    1. Profit target: Exit when option price <= premium * exit_pct
       - Uses probabilistic fill model: touch does not guarantee fill
       - Loops through days until filled or expired
    2. Stop-loss: Exit immediately when high >= stop threshold (p_fill=1.0)
    3. Expiration: If no exit by expiration, option expires worthless

    Args:
        backtest_candidates: DataFrame with options to backtest
        client: Databento client
        config: Configuration dict

    Returns:
        DataFrame with exit results
    """
    import numpy as np
    
    # Initialize RNG for reproducible fills
    rng = np.random.RandomState(config.get('execution_seed', 42))
    exits = []

    for idx, row in backtest_candidates.iterrows():
        symbol = row['symbol']

                # Generate wheel_id for this candidate
        wheel_id = generate_wheel_id()  # ADD THIS
        initial_capital = row['cost_basis']  # ADD THIS (cost_basis = strike * 100)

        # Normalize dates
        entry_date = pd.Timestamp(row['date']).tz_localize(None)
        expiration_date = pd.Timestamp(row['expiration']).tz_localize(None)

        # Entry details - work with per-share prices for comparison, contract prices for P&L
        premium_per_share = row['entry_price']  # Use entry_price (with slippage) not mid
        premium = premium_per_share * 100  # Contract premium (100 shares per contract)
        exit_pct = row['exit_pct']
        exit_price_per_share = premium_per_share * exit_pct  # Per-share exit price (buy back at this price)
        stop_loss_per_share = premium_per_share * config.get('stop_loss_multiplier', 2.0)
        cost_basis = row['strike'] * 100  # Contract cost basis
        liquidity_penalty = row.get('liquidity_penalty', 1.0)
        
        # Entry-time liquidity data for fill probability
        spread_pct_entry = row.get('spread_pct_entry', 0.05)
        ivp_entry = row.get('ivp_entry', 0.5)

        print(f"\nProcessing {symbol}...")
        print(f"  Entry: {entry_date.date()}, Premium: ${premium:.2f} (${premium_per_share:.2f}/share)")
        print(f"  Exit target: ${exit_price_per_share*100:.2f} (${exit_price_per_share:.2f}/share, exit at {exit_pct*100:.0f}% of premium)")
        print(f"  Stop loss: ${stop_loss_per_share*100:.2f} (${stop_loss_per_share:.2f}/share)")

        try:
            # Fetch daily prices
            df_daily = fetch_daily_prices_for_option(symbol, entry_date, expiration_date, client, config)

            # Initialize tracking variables
            touch_count = 0
            touch_profit_target = False
            filled_profit_target = False
            p_fill_profit = None
            u_fill_profit = None
            exit_date = None
            exit_daily_row = None
            exit_reason = None
            exit_price = None

            # Compute fill probability once (based on entry-time liquidity)
            if config.get('use_probabilistic_exit_fills', True):
                p_fill_profit = compute_p_fill_profit(row, config)

            # Loop through trading days until expiration
            for check_date, daily_row in df_daily.iterrows():
                # Normalize check_date
                check_date_normalized = check_date.tz_localize(None) if hasattr(check_date, 'tz_localize') and check_date.tz else check_date
                
                # Skip entry date
                if check_date_normalized.date() <= entry_date.date():
                    continue

                daily_low = daily_row['low']
                daily_high = daily_row['high']

                # Check stop-loss first (always fills with p_fill=1.0)
                if daily_high >= stop_loss_per_share:
                    exit_date = check_date_normalized
                    exit_daily_row = daily_row
                    exit_reason = 'stop_loss'
                    # Stop-loss: calculate exit price with slippage
                    actual_exit_per_share = get_exit_price(daily_row, config.get('fill_mode', 'realistic'), penalty=liquidity_penalty)
                    exit_price = actual_exit_per_share * 100
                    filled_profit_target = False  # Stop-loss is not a profit target fill
                    print(f"  ⚠ Stop-loss triggered on {exit_date.date()}")
                    print(f"    Threshold: ${stop_loss_per_share:.2f}/share, Actual fill: ${actual_exit_per_share:.2f}/share")
                    break

                # Check profit target touch
                if daily_low <= exit_price_per_share:
                    touch_profit_target = True
                    touch_count += 1

                    if config.get('use_probabilistic_exit_fills', True) and p_fill_profit is not None:
                        # Probabilistic fill: draw random number
                        filled, u = try_probabilistic_fill(p_fill_profit, rng)
                        u_fill_profit = u

                        if filled:
                            # Fill successful - exit trade
                            exit_date = check_date_normalized
                            exit_daily_row = daily_row
                            exit_reason = 'profit_target'
                            filled_profit_target = True
                            # Calculate exit price with slippage
                            actual_exit_per_share = get_exit_price(daily_row, config.get('fill_mode', 'realistic'), penalty=liquidity_penalty)
                            exit_price = actual_exit_per_share * 100
                            print(f"  ✓ Profit target hit on {exit_date.date()} (touch #{touch_count}, filled)")
                            print(f"    Target: ${exit_price_per_share:.2f}/share, Actual fill: ${actual_exit_per_share:.2f}/share")
                            print(f"    p_fill={p_fill_profit:.2f}, u={u:.3f}")
                            break
                        else:
                            # Touch but no fill - continue holding
                            print(f"    Touch #{touch_count} on {check_date_normalized.date()}: NO FILL (p_fill={p_fill_profit:.2f}, u={u:.3f})")
                    else:
                        # Deterministic fill (legacy behavior)
                        exit_date = check_date_normalized
                        exit_daily_row = daily_row
                        exit_reason = 'profit_target'
                        filled_profit_target = True
                        touch_count = 1
                        actual_exit_per_share = get_exit_price(daily_row, config.get('fill_mode', 'realistic'), penalty=liquidity_penalty)
                        exit_price = actual_exit_per_share * 100
                        print(f"  ✓ Profit target hit on {exit_date.date()} (deterministic)")
                        break

            # Handle expiration if no exit occurred
            if exit_date is None:
                exit_date = expiration_date
                exit_daily_row = None
                exit_reason = 'expired_worthless'
                exit_price = 0.0
                print(f"  🎉 Option expired worthless on {expiration_date.date()} - KEEP 100% PREMIUM!")

            # Create exit record with all tracking fields
            exit_record = create_exit_record(
                symbol, entry_date, expiration_date, premium, exit_pct,
                exit_price, exit_reason, exit_date, exit_daily_row, cost_basis,
                wheel_id=wheel_id, 
                initial_capital=initial_capital,
                touch_profit_target=touch_profit_target,
                p_fill_profit_target=p_fill_profit,
                u_fill_profit_target=u_fill_profit,
                filled_profit_target=filled_profit_target,
                spread_pct_entry=spread_pct_entry,
                ivp_entry=ivp_entry,
                touch_count=touch_count if touch_profit_target else 0
            )
            exits.append(exit_record)

        except Exception as e:
            print(f"  ✗ Error: {e}")
            import traceback
            traceback.print_exc()
            continue

    # Create results DataFrame and calculate P&L
    exits_df = pd.DataFrame(exits)
    exits_df = calculate_pnl_metrics(exits_df, config)

    return exits_df

# Run backtest (uses CONFIG from top of notebook)
exits_df = backtest_exit_strategy(
    backtest_candidates=backtest_candidates,
    client=client,
    config=CONFIG
)

# Display results
print("\n" + "="*60)
print("BACKTEST RESULTS")
print("="*60)
print(f"\nTotal exits: {len(exits_df)}")

if len(exits_df) > 0:
    print(f"\nExit reasons:")
    print(exits_df['exit_reason'].value_counts())
    print(f"\nP&L Summary:")
    print(exits_df[['exit_pnl', 'exit_pnl_pct', 'roc']].describe())
    
    # Show sample
    print("\nSample exits:")
    print(exits_df[['symbol', 'entry_date', 'exit_date', 'premium', 'exit_price', 
                   'exit_pnl', 'roc', 'exit_reason']].head(10))
else:
    print("\n⚠ No exits recorded - check for errors above")


NameError: name 'backtest_candidates' is not defined

In [28]:
# =============================================================================
# WHEEL MODULE: ASSIGNMENT HANDLER AND COVERED CALL SELECTION
# =============================================================================

def handle_assignment(csp_exit_record):
    """
    Create assignment record when CSP expires ITM.
    
    This captures the state when a put is assigned and shares are received.
    
    NOTE: premium_kept is recorded for audit trail but is already 
    included in CSP P&L. Do not double-count in wheel totals.
    
    Args:
        csp_exit_record: Dict from create_exit_record with exit_reason='assigned'
    
    Returns:
        dict: Assignment record with stock position details
    """
    strike = csp_exit_record['strike']
    premium = csp_exit_record['premium']
    net_stock_cost = strike * 100 - premium
    
    return {
        'wheel_id': csp_exit_record['wheel_id'],
        'symbol': csp_exit_record['symbol'].split()[0],  # Underlying (e.g., "TSLA")
        'assignment_date': csp_exit_record['expiration'],
        'strike': strike,
        'shares': 100,
        'assigned_price': strike,
        'cash_used': strike * 100,
        'premium_kept': premium,  # Audit only - already in CSP P&L
        'net_stock_cost': net_stock_cost,
        'stock_cost_per_share': net_stock_cost / 100,  # Derived field for CC strike constraint
        'underlying_at_assignment': csp_exit_record.get('underlying_at_expiration'),
        'initial_capital': csp_exit_record.get('initial_capital'),
    }


def fetch_option_chain_for_cc(underlying_symbol, entry_date, client, config, cc_config):
    """
    Fetch call option chain for covered call selection.
    
    Args:
        underlying_symbol: Underlying symbol (e.g., 'TSLA')
        entry_date: Date to enter CC position
        client: Databento client
        config: Main CONFIG dict
        cc_config: CC_CONFIG dict
    
    Returns:
        DataFrame with call options meeting criteria, or empty DataFrame
    """
    from py_vollib.black_scholes.implied_volatility import implied_volatility
    from py_vollib.black_scholes.greeks.analytical import delta as calc_delta
    import numpy as np
    
    cache_dir = config.get('cache_dir', '../cache/')
    tz = config.get('timezone', 'America/New_York')
    entry_time = cc_config.get('entry_time', '15:45')
    
    # Build entry timestamp
    entry_ts = pd.Timestamp(f"{entry_date.date()} {entry_time}", tz=tz)
    
    # Cache filename
    date_str = entry_ts.strftime('%Y%m%d')
    time_str = entry_ts.strftime('%H%M')
    cache_file = os.path.join(cache_dir, f"options_{underlying_symbol}_{date_str}_{time_str}.parquet")
    
    # Check cache first
    if os.path.exists(cache_file):
        print(f"    [CACHE HIT] Loading options for CC selection on {entry_date.date()}")
        df_opts = pd.read_parquet(cache_file)
    else:
        print(f"    [API] Fetching options for CC selection on {entry_date.date()}...")
        start = entry_ts
        end = start + pd.Timedelta(minutes=1)
        
        try:
            data = client.timeseries.get_range(
                dataset='OPRA.PILLAR',
                schema='cmbp-1',
                symbols=f"{underlying_symbol}.OPT",
                stype_in='parent',
                start=start,
                end=end,
            )
            df_opts = data.to_df(tz=tz).sort_values("ts_event")
            df_opts.to_parquet(cache_file)
        except Exception as e:
            print(f"    Error fetching options: {e}")
            return pd.DataFrame()
    
    if len(df_opts) == 0:
        return pd.DataFrame()
    
    # Parse option symbols
    sym = df_opts["symbol"]
    root_and_code = sym.str.split(expand=True)
    df_opts["root"] = root_and_code[0]
    code = root_and_code[1]
    df_opts["expiration"] = pd.to_datetime(code.str[:6], format="%y%m%d")
    df_opts["call_put"] = code.str[6]
    strike_int = code.str[7:].astype("int32")
    df_opts["strike"] = strike_int / 1000.0
    expiration_tz = df_opts["expiration"].dt.tz_localize(df_opts["ts_event"].dt.tz)
    df_opts["dte"] = (expiration_tz - df_opts["ts_event"].dt.normalize()).dt.days
    
    # Filter to calls only with valid quotes
    calls = df_opts[
        (df_opts['call_put'] == 'C') &
        (df_opts['bid_px_00'].notna()) &
        (df_opts['ask_px_00'].notna()) &
        (df_opts['dte'] >= cc_config['dte_min']) &
        (df_opts['dte'] <= cc_config['dte_max'])
    ].copy()
    
    if len(calls) == 0:
        return pd.DataFrame()
    
    # Compute mid price
    calls['mid'] = (calls['bid_px_00'] + calls['ask_px_00']) / 2
    calls['spread'] = calls['ask_px_00'] - calls['bid_px_00']
    calls['spread_pct'] = calls['spread'] / calls['mid']
    
    # Get last quote per contract
    calls = calls.sort_values("ts_event").groupby(
        ["symbol", "expiration", "strike", "call_put"]
    ).tail(1).copy()
    
    # Get underlying price
    equity_cache = os.path.join(cache_dir, f"equity_minute_{underlying_symbol}_{date_str}_{time_str}.parquet")
    if os.path.exists(equity_cache):
        equity_df = pd.read_parquet(equity_cache)
        underlying_price = equity_df['close'].iloc[-1] if len(equity_df) > 0 else None
    else:
        underlying_price = None
    
    if underlying_price is None:
        print(f"    Warning: Could not get underlying price for delta calc")
        return pd.DataFrame()
    
    calls['underlying_last'] = underlying_price
    
    # Compute IV and delta
    r = 0.04  # Risk-free rate
    
    def compute_iv_delta(row):
        price = row['mid']
        S = row['underlying_last']
        K = row['strike']
        t = row['dte'] / 365.0
        
        if not (np.isfinite(price) and np.isfinite(S) and np.isfinite(K) and t > 0):
            return np.nan, np.nan
        if price <= 0 or S <= 0 or K <= 0:
            return np.nan, np.nan
        
        try:
            iv = implied_volatility(price, S, K, t, r, 'c')
            d = abs(calc_delta('c', S, K, t, r, iv))
            return iv, d
        except Exception:
            return np.nan, np.nan
    
    iv_delta = calls.apply(compute_iv_delta, axis=1, result_type='expand')
    calls['iv'] = iv_delta[0]
    calls['delta'] = iv_delta[1]
    
    # Filter by delta
    calls = calls[
        calls['delta'].between(cc_config['delta_min'], cc_config['delta_max'])
    ].copy()
    
    return calls


def select_covered_call(assignment_record, option_chain, cc_config, config):
    """
    Select covered call to sell after assignment.
    
    Args:
        assignment_record: Dict from handle_assignment()
        option_chain: DataFrame with call options (from fetch_option_chain_for_cc)
        cc_config: CC_CONFIG dict
        config: Main CONFIG dict
    
    Returns:
        Series with selected call, or None if no suitable call found
    """
    if len(option_chain) == 0:
        return None
    
    stock_cost_per_share = assignment_record['stock_cost_per_share']
    
    # Filter by strike constraint if enabled
    if cc_config.get('sell_call_only_if_price_above_basis', True):
        candidates = option_chain[
            option_chain['strike'] >= stock_cost_per_share
        ].copy()
    else:
        candidates = option_chain.copy()
    
    if len(candidates) == 0:
        print(f"    No calls with strike >= cost basis ${stock_cost_per_share:.2f}")
        return None
    
    # Apply liquidity model
    candidates = apply_liquidity_model(candidates, config)
    
    if len(candidates) == 0:
        print(f"    No calls passed liquidity filter")
        return None
    
    # Tie-breaking
    method = cc_config.get('tie_break_method', 'highest_premium')
    
    if method == 'highest_premium':
        selected = candidates.loc[candidates['mid'].idxmax()]
    elif method == 'closest_delta':
        target_delta = (cc_config['delta_min'] + cc_config['delta_max']) / 2
        candidates['delta_dist'] = (candidates['delta'].abs() - target_delta).abs()
        selected = candidates.loc[candidates['delta_dist'].idxmin()]
    elif method == 'highest_strike':
        selected = candidates.loc[candidates['strike'].idxmax()]
    else:
        selected = candidates.iloc[0]  # Fallback
    
    print(f"    Selected CC: {selected['symbol']}")
    print(f"      Strike: ${selected['strike']:.2f}, Delta: {selected['delta']:.3f}, Premium: ${selected['mid']:.2f}/share")
    print(f"      DTE: {selected['dte']}, Expiration: {selected['expiration'].date()}")
    
    return selected


print("=" * 60)
print("WHEEL MODULE: Assignment and Covered Call Functions Loaded")
print("=" * 60)
print("  - handle_assignment(): Create assignment record from CSP exit")
print("  - fetch_option_chain_for_cc(): Fetch call chain for CC selection")
print("  - select_covered_call(): Select optimal call to sell")
print("=" * 60)



WHEEL MODULE: Assignment and Covered Call Functions Loaded
  - handle_assignment(): Create assignment record from CSP exit
  - fetch_option_chain_for_cc(): Fetch call chain for CC selection
  - select_covered_call(): Select optimal call to sell


In [29]:

# =============================================================================
# COVERED CALL BACKTEST FUNCTION
# =============================================================================

def backtest_covered_call(assignment_record, cc_selection, client, config, cc_config):
    """
    Backtest a covered call position after assignment.
    
    Uses same exit framework as CSP with CC-specific adjustments:
    - Profit target: Buy back call at cc_config exit_pct
    - Stop loss: Not typically used for CC (we own the shares)
    - Called away: Underlying >= strike at expiration
    - Expired worthless: Underlying < strike at expiration
    
    Args:
        assignment_record: Dict from handle_assignment()
        cc_selection: Series from select_covered_call()
        client: Databento client
        config: Main CONFIG dict
        cc_config: CC_CONFIG dict
    
    Returns:
        dict: Exit record for covered call phase
    """
    import numpy as np
    
    wheel_id = assignment_record['wheel_id']
    underlying_symbol = assignment_record['symbol']
    initial_capital = assignment_record['initial_capital']
    net_stock_cost = assignment_record['net_stock_cost']
    
    symbol = cc_selection['symbol']
    strike = cc_selection['strike']
    expiration_date = pd.Timestamp(cc_selection['expiration']).tz_localize(None)
    
    # Entry details
    entry_date = assignment_record['assignment_date']
    if hasattr(entry_date, 'tz') and entry_date.tz:
        entry_date = entry_date.tz_localize(None)
    
    # CC entry is next trading day after assignment
    # For simplicity, use assignment date as CC entry (could be refined with market calendar)
    cc_entry_date = pd.Timestamp(entry_date) + pd.Timedelta(days=1)
    cc_entry_date = cc_entry_date.tz_localize(None)
    
    # Calculate entry price with liquidity penalty
    liquidity_penalty = cc_selection.get('liquidity_penalty', 1.0)
    premium_per_share = get_entry_price(cc_selection, config['fill_mode'], liquidity_penalty)
    premium = premium_per_share * 100
    cost_basis = strike * 100  # CC cost basis is the strike
    
    # Exit parameters (use main config for exit_pct)
    exit_pct = config.get('exit_pct', 0.50)
    exit_price_per_share = premium_per_share * exit_pct
    
    # Initialize state
    current_state = advance_wheel_state('CSP_ASSIGNED', 'sell_call')  # CSP_ASSIGNED → CC_OPEN
    
    print(f"\n  CC Position: {symbol}")
    print(f"    Wheel ID: {wheel_id}, State: {current_state}")
    print(f"    Entry: {cc_entry_date.date()}, Strike: ${strike:.2f}, Premium: ${premium:.2f}")
    print(f"    Exit target: ${exit_price_per_share*100:.2f} (${exit_price_per_share:.2f}/share)")
    
    # Initialize RNG
    rng = np.random.RandomState(config.get('execution_seed', 42) + hash(wheel_id) % 1000)
    
    # Tracking variables
    touch_count = 0
    touch_profit_target = False
    filled_profit_target = False
    p_fill_profit = None
    u_fill_profit = None
    exit_date = None
    exit_daily_row = None
    exit_reason = None
    exit_price = None
    underlying_at_exp = None
    
    try:
        # Fetch daily prices for the call option
        df_daily = fetch_daily_prices_for_option(symbol, cc_entry_date, expiration_date, client, config)
        
        # Compute fill probability
        if config.get('use_probabilistic_exit_fills', True):
            p_fill_profit = compute_p_fill_profit(cc_selection, config)
        
        # Loop through trading days
        for check_date, daily_row in df_daily.iterrows():
            check_date_normalized = check_date.tz_localize(None) if hasattr(check_date, 'tz_localize') and check_date.tz else check_date
            
            if check_date_normalized.date() <= cc_entry_date.date():
                continue
            
            daily_low = daily_row['low']
            daily_high = daily_row['high']
            
            # Check profit target (call decays)
            if daily_low <= exit_price_per_share:
                touch_profit_target = True
                touch_count += 1
                
                if config.get('use_probabilistic_exit_fills', True) and p_fill_profit is not None:
                    filled, u = try_probabilistic_fill(p_fill_profit, rng)
                    u_fill_profit = u
                    
                    if filled:
                        exit_date = check_date_normalized
                        exit_daily_row = daily_row
                        exit_reason = 'profit_target'
                        current_state = advance_wheel_state(current_state, 'profit_target')
                        filled_profit_target = True
                        actual_exit_per_share = get_exit_price(daily_row, config.get('fill_mode', 'realistic'), penalty=liquidity_penalty)
                        exit_price = actual_exit_per_share * 100
                        print(f"    ✓ CC profit target on {exit_date.date()} → State: {current_state}")
                        break
                else:
                    exit_date = check_date_normalized
                    exit_daily_row = daily_row
                    exit_reason = 'profit_target'
                    current_state = advance_wheel_state(current_state, 'profit_target')
                    filled_profit_target = True
                    actual_exit_per_share = get_exit_price(daily_row, config.get('fill_mode', 'realistic'), penalty=liquidity_penalty)
                    exit_price = actual_exit_per_share * 100
                    print(f"    ✓ CC profit target on {exit_date.date()} (deterministic) → State: {current_state}")
                    break
        
        # Handle expiration - check for assignment
        if exit_date is None:
            exit_date = expiration_date
            exit_daily_row = None
            exit_price = 0.0
            
            # Fetch underlying price at expiration
            underlying_at_exp = fetch_underlying_price_at_expiration(
                underlying_symbol, expiration_date, client, config
            )
            
            if underlying_at_exp is not None:
                # For calls: ITM when underlying >= strike → called away
                if underlying_at_exp >= strike:
                    exit_reason = 'called_away'
                    current_state = advance_wheel_state(current_state, 'called_away')
                    print(f"    📤 CC CALLED AWAY on {expiration_date.date()} → State: {current_state}")
                    print(f"      Underlying: ${underlying_at_exp:.2f} >= Strike: ${strike:.2f}")
                else:
                    exit_reason = 'expired_worthless'
                    current_state = advance_wheel_state(current_state, 'expired_worthless')
                    print(f"    🎉 CC expired worthless on {expiration_date.date()} → State: {current_state}")
            else:
                exit_reason = 'expired_worthless'
                current_state = advance_wheel_state(current_state, 'expired_worthless')
                print(f"    CC expired (assumed worthless) → State: {current_state}")
    
    except Exception as e:
        print(f"    ✗ CC Error: {e}")
        import traceback
        traceback.print_exc()
        exit_date = expiration_date
        exit_reason = 'error'
        exit_price = 0.0
        current_state = 'CC_CLOSED_WORTHLESS'
    
    # Create exit record
    cc_exit_record = {
        'wheel_id': wheel_id,
        'phase': 'cc',
        'state': current_state,
        'symbol': symbol,
        'strike': strike,
        'entry_date': cc_entry_date,
        'exit_date': exit_date,
        'expiration': expiration_date,
        'cost_basis': cost_basis,
        'initial_capital': initial_capital,
        'premium': premium,
        'exit_pct': exit_pct,
        'exit_price': exit_price,
        'exit_reason': exit_reason,
        'days_held': (exit_date - cc_entry_date).days if exit_date else None,
        'underlying_at_expiration': underlying_at_exp,
        'daily_low': exit_daily_row['low'] if exit_daily_row is not None else None,
        'daily_high': exit_daily_row['high'] if exit_daily_row is not None else None,
        'touch_profit_target': touch_profit_target,
        'p_fill_profit_target': p_fill_profit,
        'u_fill_profit_target': u_fill_profit,
        'filled_profit_target': filled_profit_target,
        'touch_count': touch_count,
        'fill_model': 'ohlc_touch_prob_v1',
        # Stock position info for P&L
        'net_stock_cost': net_stock_cost,
    }
    
    return cc_exit_record, current_state


print("=" * 60)
print("COVERED CALL BACKTEST FUNCTION LOADED")
print("=" * 60)


COVERED CALL BACKTEST FUNCTION LOADED


In [30]:
# =============================================================================
# WHEEL ORCHESTRATION AND P&L CALCULATION
# =============================================================================

def calculate_wheel_pnl(csp_exit, cc_exit=None, assignment_record=None):
    """
    Calculate complete wheel P&L including CSP, CC, and stock legs.
    
    P&L Components:
    - CSP P&L: premium - exit_price - fees (already calculated)
    - CC P&L: call_premium - exit_price - call_fees
    - Stock P&L: (call_strike * 100) - net_stock_cost (only if called away)
    
    Wheel-level ROC is based on initial CSP capital (strike * 100).
    
    Args:
        csp_exit: CSP exit record dict
        cc_exit: CC exit record dict (optional, only if assigned)
        assignment_record: Assignment record dict (optional, only if assigned)
    
    Returns:
        dict: Wheel summary with total P&L
    """
    wheel_id = csp_exit['wheel_id']
    initial_capital = csp_exit.get('initial_capital', csp_exit['cost_basis'])
    
    # CSP P&L
    csp_premium = csp_exit['premium']
    csp_exit_price = csp_exit['exit_price']
    csp_fees = get_transaction_costs(CONFIG, is_round_trip=(csp_exit['exit_reason'] not in ['expired_worthless', 'assigned']))
    csp_pnl = csp_premium - csp_exit_price - csp_fees
    
    # Initialize totals
    cc_pnl = 0.0
    stock_pnl = 0.0
    cc_fees = 0.0
    total_days = csp_exit.get('days_held', 0) or 0
    
    # CC P&L (if applicable)
    if cc_exit is not None:
        cc_premium = cc_exit['premium']
        cc_exit_price = cc_exit['exit_price']
        cc_fees = get_transaction_costs(CONFIG, is_round_trip=(cc_exit['exit_reason'] not in ['expired_worthless', 'called_away']))
        cc_pnl = cc_premium - cc_exit_price - cc_fees
        total_days += cc_exit.get('days_held', 0) or 0
        
        # Stock P&L (only if called away)
        if cc_exit['exit_reason'] == 'called_away' and assignment_record is not None:
            net_stock_cost = assignment_record['net_stock_cost']
            call_strike = cc_exit['strike']
            stock_pnl = (call_strike * 100) - net_stock_cost
    
    # Total wheel P&L
    total_pnl = csp_pnl + cc_pnl + stock_pnl
    total_fees = csp_fees + cc_fees
    
    # ROC calculations
    csp_roc = (csp_pnl / initial_capital) * 100
    wheel_roc = (total_pnl / initial_capital) * 100
    
    # Determine final state using state machine
    if cc_exit is not None:
        # CC was processed - use CC's final state or advance to complete
        cc_state = cc_exit.get('state', 'CC_ASSIGNED')
        if cc_state == 'WHEEL_COMPLETE':
            final_state = 'WHEEL_COMPLETE'
        else:
            final_state = advance_wheel_state(cc_state, 'complete')
    else:
        # No CC processed - advance CSP state to complete
        # This works for both assigned (incomplete) and non-assigned CSPs
        final_state = advance_wheel_state(csp_exit['state'], 'complete')
    
    return {
        'wheel_id': wheel_id,
        'phase': 'total',
        'state': final_state,
        'initial_capital': initial_capital,
        
        # CSP details
        'csp_premium': csp_premium,
        'csp_exit_price': csp_exit_price,
        'csp_pnl': csp_pnl,
        'csp_fees': csp_fees,
        
        # CC details
        'cc_premium': cc_exit['premium'] if cc_exit else 0.0,
        'cc_exit_price': cc_exit['exit_price'] if cc_exit else 0.0,
        'cc_pnl': cc_pnl,
        'cc_fees': cc_fees,
        
        # Stock details
        'stock_pnl': stock_pnl,
        
        # Totals
        'total_pnl': total_pnl,
        'total_fees': total_fees,
        'csp_roc': csp_roc,
        'wheel_roc': wheel_roc,
        'total_days': total_days,
        
        # Exit info
        'csp_exit_reason': csp_exit['exit_reason'],
        'cc_exit_reason': cc_exit['exit_reason'] if cc_exit else None,
    }


def run_full_wheel_backtest(csp_exits_df, client, config, cc_config):
    """
    Run full wheel backtest: process CSP exits and handle assignments with CC.
    
    For each CSP that was assigned:
    1. Create assignment record
    2. Fetch call chain for next trading day
    3. Select covered call
    4. Backtest CC position
    5. Calculate wheel P&L
    
    Args:
        csp_exits_df: DataFrame with CSP exit records
        client: Databento client
        config: Main CONFIG dict
        cc_config: CC_CONFIG dict
    
    Returns:
        tuple: (all_exits_df, wheel_summaries_df)
    """
    all_exits = []
    wheel_summaries = []
    
    for idx, csp_exit in csp_exits_df.iterrows():
        # Convert row to dict
        csp_exit_dict = csp_exit.to_dict()
        all_exits.append(csp_exit_dict)
        
        # Check if assigned
        if csp_exit['exit_reason'] != 'assigned':
            # No assignment - calculate simple wheel P&L (CSP only)
            wheel_summary = calculate_wheel_pnl(csp_exit_dict)
            wheel_summaries.append(wheel_summary)
            print(f"\n  Wheel {csp_exit['wheel_id']}: CSP {csp_exit['exit_reason']} → {wheel_summary['state']}")
            print(f"    CSP P&L: ${wheel_summary['csp_pnl']:.2f}, ROC: {wheel_summary['csp_roc']:.2f}%")
            continue
        
        # Handle assignment
        print(f"\n{'='*60}")
        print(f"PROCESSING ASSIGNMENT: Wheel {csp_exit['wheel_id']}")
        print(f"{'='*60}")
        
        assignment_record = handle_assignment(csp_exit_dict)
        print(f"  Assignment: {assignment_record['shares']} shares at ${assignment_record['strike']:.2f}")
        print(f"  Net stock cost: ${assignment_record['net_stock_cost']:.2f} (${assignment_record['stock_cost_per_share']:.2f}/share)")
        
        # Fetch call chain for CC selection (next trading day)
        underlying_symbol = assignment_record['symbol']
        cc_entry_date = pd.Timestamp(assignment_record['assignment_date']) + pd.Timedelta(days=1)
        
        call_chain = fetch_option_chain_for_cc(
            underlying_symbol, cc_entry_date, client, config, cc_config
        )
        
        if len(call_chain) == 0:
            print(f"    ⚠ No suitable calls found - wheel incomplete")
            wheel_summary = calculate_wheel_pnl(csp_exit_dict)
            wheel_summaries.append(wheel_summary)
            continue
        
        # Select covered call
        cc_selection = select_covered_call(assignment_record, call_chain, cc_config, config)
        
        if cc_selection is None:
            print(f"    ⚠ No call selected - wheel incomplete")
            wheel_summary = calculate_wheel_pnl(csp_exit_dict)
            wheel_summaries.append(wheel_summary)
            continue
        
        # Backtest covered call
        cc_exit_dict, final_cc_state = backtest_covered_call(
            assignment_record, cc_selection, client, config, cc_config
        )
        all_exits.append(cc_exit_dict)
        
        # Calculate complete wheel P&L
        wheel_summary = calculate_wheel_pnl(csp_exit_dict, cc_exit_dict, assignment_record)
        wheel_summaries.append(wheel_summary)
        
        print(f"\n  WHEEL COMPLETE: {wheel_summary['wheel_id']}")
        print(f"    CSP P&L:   ${wheel_summary['csp_pnl']:.2f}")
        print(f"    CC P&L:    ${wheel_summary['cc_pnl']:.2f}")
        print(f"    Stock P&L: ${wheel_summary['stock_pnl']:.2f}")
        print(f"    ────────────────────────")
        print(f"    TOTAL:     ${wheel_summary['total_pnl']:.2f} ({wheel_summary['wheel_roc']:.2f}% ROC)")
        print(f"    Days:      {wheel_summary['total_days']}")
    
    # Create DataFrames
    all_exits_df = pd.DataFrame(all_exits)
    wheel_summaries_df = pd.DataFrame(wheel_summaries)
    
    return all_exits_df, wheel_summaries_df


print("=" * 60)
print("WHEEL ORCHESTRATION FUNCTIONS LOADED")
print("=" * 60)
print("  - calculate_wheel_pnl(): Calculate complete wheel P&L")
print("  - run_full_wheel_backtest(): Process CSP exits and handle CCs")
print("=" * 60)


WHEEL ORCHESTRATION FUNCTIONS LOADED
  - calculate_wheel_pnl(): Calculate complete wheel P&L
  - run_full_wheel_backtest(): Process CSP exits and handle CCs


In [40]:
# =============================================================================
# V5 SINGLE WHEEL WRAPPER
# =============================================================================
# Wraps the existing wheel engine to process a single candidate

def run_single_wheel(candidate, config, cc_config, wheel_id, client):
    """
    Run a complete wheel cycle for a single CSP candidate.
    
    This wrapper:
    1. Calls backtest_exit_strategy() for CSP lifecycle
    2. If assigned, handles CC phase via existing functions
    3. Calculates wheel P&L
    4. Stamps results with execution_version for downstream tracking
    
    Args:
        candidate: Series with CSP candidate data (from get_entry_candidates)
        config: CONFIG dict
        cc_config: CC_CONFIG dict
        wheel_id: Unique identifier for this wheel instance
        client: Databento client
    
    Returns:
        List[dict]: Exit records for this wheel (CSP + optional CC + summary)
    """
    import numpy as np
    
    # Convert candidate Series to single-row DataFrame for backtest_exit_strategy
    candidate_df = pd.DataFrame([candidate])
    
    # Initialize RNG with unique seed per wheel
    wheel_seed = config.get('execution_seed', 42) + hash(wheel_id) % 10000
    
    # Create a copy of config with wheel-specific seed
    wheel_config = config.copy()
    wheel_config['execution_seed'] = wheel_seed
    
    # Run CSP backtest (silent version)
    csp_exit = run_csp_backtest_silent(candidate_df, client, wheel_config, wheel_id)
    
    if csp_exit is None:
        return []
    
    wheel_results = []
    
    # Add CSP exit with phase marker
    csp_exit['phase'] = 'csp'
    csp_exit['execution_version'] = 'v5_scheduler'
    wheel_results.append(csp_exit)
    
    # Check if assigned
    if csp_exit['exit_reason'] == 'assigned':
        # Handle assignment and CC
        assignment_record = handle_assignment(csp_exit)
        
        # Fetch call chain
        underlying_symbol = assignment_record['symbol']
        cc_entry_date = pd.Timestamp(assignment_record['assignment_date']) + pd.Timedelta(days=1)
        
        call_chain = fetch_option_chain_for_cc(
            underlying_symbol, cc_entry_date, client, config, cc_config
        )
        
        if len(call_chain) > 0:
            cc_selection = select_covered_call(assignment_record, call_chain, cc_config, config)
            
            if cc_selection is not None:
                cc_exit, _ = backtest_covered_call(
                    assignment_record, cc_selection, client, wheel_config, cc_config
                )
                cc_exit['execution_version'] = 'v5_scheduler'
                wheel_results.append(cc_exit)
                
                # Calculate wheel P&L with CC
                wheel_summary = calculate_wheel_pnl(csp_exit, cc_exit, assignment_record)
            else:
                # No CC selected
                wheel_summary = calculate_wheel_pnl(csp_exit)
        else:
            # No call chain available
            wheel_summary = calculate_wheel_pnl(csp_exit)
    else:
        # Not assigned - simple CSP wheel
        wheel_summary = calculate_wheel_pnl(csp_exit)
    
    # Add summary with execution version
    wheel_summary['execution_version'] = 'v5_scheduler'
    wheel_results.append(wheel_summary)
    
    return wheel_results


def run_csp_backtest_silent(backtest_candidates, client, config, wheel_id):
    """
    Run CSP backtest for a single candidate, returning dict instead of DataFrame.
    Silent version for scheduler use.
    
    Args:
        backtest_candidates: Single-row DataFrame with CSP candidate
        client: Databento client
        config: CONFIG dict
        wheel_id: Wheel identifier
    
    Returns:
        dict: Exit record for the CSP, or None if error
    """
    import numpy as np
    
    rng = np.random.RandomState(config.get('execution_seed', 42))
    
    for idx, row in backtest_candidates.iterrows():
        symbol = row['symbol']
        initial_capital = row['cost_basis']
        
        # Normalize dates
        entry_date = pd.Timestamp(row['date']).tz_localize(None)
        expiration_date = pd.Timestamp(row['expiration']).tz_localize(None)
        
        # Entry details
        premium_per_share = row['entry_price']
        premium = premium_per_share * 100
        exit_pct = row['exit_pct']
        exit_price_per_share = premium_per_share * exit_pct
        stop_loss_per_share = premium_per_share * config.get('stop_loss_multiplier', 2.0)
        cost_basis = row['strike'] * 100
        liquidity_penalty = row.get('liquidity_penalty', 1.0)
        spread_pct_entry = row.get('spread_pct_entry', 0.05)
        ivp_entry = row.get('ivp_entry', 0.5)
        
        try:
            df_daily = fetch_daily_prices_for_option(symbol, entry_date, expiration_date, client, config)
            
            touch_count = 0
            touch_profit_target = False
            filled_profit_target = False
            p_fill_profit = None
            u_fill_profit = None
            exit_date = None
            exit_daily_row = None
            exit_reason = None
            exit_price = None
            
            if config.get('use_probabilistic_exit_fills', True):
                p_fill_profit = compute_p_fill_profit(row, config)
            
            for check_date, daily_row in df_daily.iterrows():
                check_date_normalized = check_date.tz_localize(None) if hasattr(check_date, 'tz_localize') and check_date.tz else check_date
                
                if check_date_normalized.date() <= entry_date.date():
                    continue
                
                daily_low = daily_row['low']
                daily_high = daily_row['high']
                
                # Stop-loss check
                if daily_high >= stop_loss_per_share:
                    exit_date = check_date_normalized
                    exit_daily_row = daily_row
                    exit_reason = 'stop_loss'
                    actual_exit_per_share = get_exit_price(daily_row, config.get('fill_mode', 'realistic'), penalty=liquidity_penalty)
                    exit_price = actual_exit_per_share * 100
                    break
                
                # Profit target check
                if daily_low <= exit_price_per_share:
                    touch_profit_target = True
                    touch_count += 1
                    
                    if config.get('use_probabilistic_exit_fills', True) and p_fill_profit is not None:
                        filled, u = try_probabilistic_fill(p_fill_profit, rng)
                        u_fill_profit = u
                        
                        if filled:
                            exit_date = check_date_normalized
                            exit_daily_row = daily_row
                            exit_reason = 'profit_target'
                            filled_profit_target = True
                            actual_exit_per_share = get_exit_price(daily_row, config.get('fill_mode', 'realistic'), penalty=liquidity_penalty)
                            exit_price = actual_exit_per_share * 100
                            break
                    else:
                        exit_date = check_date_normalized
                        exit_daily_row = daily_row
                        exit_reason = 'profit_target'
                        filled_profit_target = True
                        actual_exit_per_share = get_exit_price(daily_row, config.get('fill_mode', 'realistic'), penalty=liquidity_penalty)
                        exit_price = actual_exit_per_share * 100
                        break
            
            # Handle expiration
            if exit_date is None:
                exit_date = expiration_date
                exit_daily_row = None
                
                # Check if assigned (ITM at expiration)
                underlying_at_exp = fetch_underlying_price_at_expiration(
                    symbol.split()[0], expiration_date, client, config
                )
                
                if underlying_at_exp is not None and underlying_at_exp < row['strike']:
                    exit_reason = 'assigned'
                    exit_price = 0.0  # Put exercised
                else:
                    exit_reason = 'expired_worthless'
                    exit_price = 0.0
            
            # Create exit record
            exit_record = create_exit_record(
                symbol, entry_date, expiration_date, premium, exit_pct,
                exit_price, exit_reason, exit_date, exit_daily_row, cost_basis,
                wheel_id=wheel_id,
                initial_capital=initial_capital,
                touch_profit_target=touch_profit_target,
                p_fill_profit_target=p_fill_profit,
                u_fill_profit_target=u_fill_profit,
                filled_profit_target=filled_profit_target,
                spread_pct_entry=spread_pct_entry,
                ivp_entry=ivp_entry,
                touch_count=touch_count if touch_profit_target else 0
            )
            
            # Add strike for assignment handling
            exit_record['strike'] = row['strike']
            
            # Calculate P&L
            is_round_trip = exit_reason not in ['expired_worthless', 'assigned']
            fees = get_transaction_costs(config, is_round_trip=is_round_trip)
            exit_record['fees'] = fees
            exit_record['exit_pnl'] = premium - exit_price - fees
            exit_record['exit_pnl_pct'] = (exit_record['exit_pnl'] / premium) * 100 if premium > 0 else 0
            exit_record['roc'] = (exit_record['exit_pnl'] / cost_basis) * 100
            
            return exit_record
            
        except Exception as e:
            return None
    
    return None


print("=" * 60)
print("V5 SINGLE WHEEL WRAPPER LOADED")
print("=" * 60)
print("  - run_single_wheel(candidate, config, cc_config, wheel_id, client)")
print("  - run_csp_backtest_silent(backtest_candidates, client, config, wheel_id)")
print("=" * 60)


V5 SINGLE WHEEL WRAPPER LOADED
  - run_single_wheel(candidate, config, cc_config, wheel_id, client)
  - run_csp_backtest_silent(backtest_candidates, client, config, wheel_id)


In [41]:
# =============================================================================
# V5 MAIN SCHEDULER LOOP
# =============================================================================

def run_v5_scheduler(CONFIG, CC_CONFIG, SCHEDULER_CONFIG, client):
    """
    Main V5 scheduler: iterate over trading dates and symbols, launch wheels.
    
    For each trading day in the date range:
        For each symbol:
            1. Get entry candidates
            2. For each candidate, run a wheel instance
            3. Collect all results
    
    Args:
        CONFIG: Main configuration dict
        CC_CONFIG: Covered call configuration dict
        SCHEDULER_CONFIG: Scheduler configuration dict
        client: Databento client
    
    Returns:
        DataFrame with all wheel results (CSP + CC + summaries)
    """
    from copy import deepcopy
    
    all_wheel_results = []
    wheel_counter = 0
    log_info = SCHEDULER_CONFIG.get('log_level', 'INFO') == 'INFO'
    
    # Get trading days
    trading_days = get_trading_days(
        SCHEDULER_CONFIG['start_date'],
        SCHEDULER_CONFIG['end_date'],
        SCHEDULER_CONFIG['trading_calendar'],
    )
    
    if log_info:
        print("=" * 60)
        print("V5 SCHEDULER STARTING")
        print("=" * 60)
        print(f"Date range: {SCHEDULER_CONFIG['start_date']} to {SCHEDULER_CONFIG['end_date']}")
        print(f"Trading days: {len(trading_days)}")
        print(f"Symbols: {SCHEDULER_CONFIG['symbols']}")
        print("=" * 60)
    
    # Track candidates per day for summary
    daily_stats = []
    
    for trade_date in trading_days:
        for symbol in SCHEDULER_CONFIG['symbols']:
            # Create day-specific config
            config_day = deepcopy(CONFIG)
            config_day['symbol'] = symbol
            config_day['entry_date'] = trade_date.strftime('%Y-%m-%d')
            
            # Get entry candidates
            entry_candidates = get_entry_candidates(symbol, trade_date, config_day, client)
            
            if len(entry_candidates) == 0:
                continue
            
            if log_info:
                print(f"[{trade_date.date()}] {symbol}: {len(entry_candidates)} candidates")
            
            daily_stats.append({
                'date': trade_date.date(),
                'symbol': symbol,
                'candidates': len(entry_candidates)
            })
            
            # Process each candidate
            for idx, candidate in entry_candidates.iterrows():
                wheel_id = f"{symbol}_{trade_date.strftime('%Y%m%d')}_{wheel_counter}"
                wheel_counter += 1
                
                wheel_results = run_single_wheel(
                    candidate, config_day, CC_CONFIG, wheel_id, client
                )
                
                # Add symbol and entry_date to each result
                for result in wheel_results:
                    result['scheduler_symbol'] = symbol
                    result['scheduler_entry_date'] = trade_date.strftime('%Y-%m-%d')
                
                all_wheel_results.extend(wheel_results)
    
    if log_info:
        print(f"\n{'='*60}")
        print(f"SCHEDULER COMPLETE")
        print(f"{'='*60}")
        print(f"Wheels launched: {wheel_counter}")
        print(f"Total result records: {len(all_wheel_results)}")
    
    # Handle empty results case
    if len(all_wheel_results) == 0:
        print("⚠ No wheel results - check date range and data availability")
        return pd.DataFrame()
    
    # Combine all results
    results_df = pd.DataFrame(all_wheel_results)
    
    return results_df


print("=" * 60)
print("V5 MAIN SCHEDULER LOADED")
print("=" * 60)
print("  - run_v5_scheduler(CONFIG, CC_CONFIG, SCHEDULER_CONFIG, client)")
print("=" * 60)


V5 MAIN SCHEDULER LOADED
  - run_v5_scheduler(CONFIG, CC_CONFIG, SCHEDULER_CONFIG, client)


In [42]:
# =============================================================================
# V5 AGGREGATION FUNCTIONS
# =============================================================================

def aggregate_v5_results(df):
    """
    Aggregate V5 scheduler results into summary statistics.
    
    Uses the 'total' phase rows which contain wheel-level P&L.
    
    Args:
        df: DataFrame from run_v5_scheduler() with all exit records
    
    Returns:
        dict: Summary statistics for the backtest
    """
    if len(df) == 0:
        return {
            'total_wheels': 0,
            'total_pnl': 0.0,
            'avg_wheel_roc': 0.0,
            'median_wheel_roc': 0.0,
            'max_drawdown_proxy': 0.0,
        }
    
    # Get wheel summary rows (phase == 'total')
    totals = df[df['phase'] == 'total'].copy()
    
    if len(totals) == 0:
        return {
            'total_wheels': 0,
            'total_pnl': 0.0,
            'avg_wheel_roc': 0.0,
            'median_wheel_roc': 0.0,
            'max_drawdown_proxy': 0.0,
        }
    
    # Use 'total_pnl' for wheel summaries, 'exit_pnl' for phase exits
    pnl_col = 'total_pnl' if 'total_pnl' in totals.columns else 'exit_pnl'
    roc_col = 'wheel_roc' if 'wheel_roc' in totals.columns else 'roc'
    
    return {
        'total_wheels': df['wheel_id'].nunique(),
        'total_pnl': totals[pnl_col].sum() if pnl_col in totals.columns else 0.0,
        'avg_wheel_roc': totals[roc_col].mean() if roc_col in totals.columns else 0.0,
        'median_wheel_roc': totals[roc_col].median() if roc_col in totals.columns else 0.0,
        'max_drawdown_proxy': totals[pnl_col].min() if pnl_col in totals.columns else 0.0,
        'win_rate': (totals[pnl_col] > 0).mean() * 100 if pnl_col in totals.columns else 0.0,
        'avg_days_held': totals['total_days'].mean() if 'total_days' in totals.columns else 0.0,
    }


def print_v5_summary(results_df, agg_stats):
    """
    Print formatted summary of V5 backtest results.
    
    Args:
        results_df: DataFrame from run_v5_scheduler()
        agg_stats: dict from aggregate_v5_results()
    """
    print("\n" + "=" * 60)
    print("V5 SCHEDULER RESULTS SUMMARY")
    print("=" * 60)
    
    print(f"\n{'─'*40}")
    print("AGGREGATE STATISTICS")
    print(f"{'─'*40}")
    print(f"Total Wheels:      {agg_stats['total_wheels']}")
    print(f"Total P&L:         ${agg_stats['total_pnl']:,.2f}")
    print(f"Avg Wheel ROC:     {agg_stats['avg_wheel_roc']:.2f}%")
    print(f"Median Wheel ROC:  {agg_stats['median_wheel_roc']:.2f}%")
    print(f"Win Rate:          {agg_stats['win_rate']:.1f}%")
    print(f"Avg Days Held:     {agg_stats['avg_days_held']:.1f}")
    print(f"Max Loss (proxy):  ${agg_stats['max_drawdown_proxy']:,.2f}")
    
    # Phase breakdown
    if 'phase' in results_df.columns:
        print(f"\n{'─'*40}")
        print("PHASE BREAKDOWN")
        print(f"{'─'*40}")
        print(results_df['phase'].value_counts().to_string())
    
    # Exit reason breakdown for CSP phase
    csp_exits = results_df[results_df['phase'] == 'csp']
    if len(csp_exits) > 0 and 'exit_reason' in csp_exits.columns:
        print(f"\n{'─'*40}")
        print("CSP EXIT REASONS")
        print(f"{'─'*40}")
        print(csp_exits['exit_reason'].value_counts().to_string())
    
    print("\n" + "=" * 60)


print("=" * 60)
print("V5 AGGREGATION FUNCTIONS LOADED")
print("=" * 60)
print("  - aggregate_v5_results(df)")
print("  - print_v5_summary(results_df, agg_stats)")
print("=" * 60)


V5 AGGREGATION FUNCTIONS LOADED
  - aggregate_v5_results(df)
  - print_v5_summary(results_df, agg_stats)


In [43]:
# =============================================================================
# V5 DATA MODEL VALIDATION
# =============================================================================

def validate_v5_results(exits_df):
    """
    Validate V5 scheduler results for data integrity.
    
    Checks:
    1. Every wheel_id has at least one phase
    2. Required fields are present
    3. Execution version is consistent
    4. Exactly one 'total' row per wheel_id
    
    Args:
        exits_df: DataFrame from run_v5_scheduler()
    
    Returns:
        bool: True if all validations pass
    
    Raises:
        AssertionError if any validation fails
    """
    if len(exits_df) == 0:
        print("⚠ Empty results - skipping validation")
        return True
    
    print("=" * 60)
    print("V5 DATA MODEL VALIDATION")
    print("=" * 60)
    
    # Check 1: Every wheel_id has at least one phase
    phase_counts = exits_df.groupby('wheel_id')['phase'].nunique()
    assert phase_counts.ge(1).all(), "Some wheel_ids have no phases"
    print("  ✓ Every wheel_id has at least one phase")
    
    # Check 2: Required fields present
    required_fields = ['wheel_id', 'phase', 'execution_version']
    missing = [col for col in required_fields if col not in exits_df.columns]
    assert len(missing) == 0, f"Missing required fields: {missing}"
    print(f"  ✓ Required fields present: {required_fields}")
    
    # Check 3: Execution version is consistent
    versions = exits_df['execution_version'].unique()
    assert len(versions) == 1 and versions[0] == 'v5_scheduler', \
        f"Inconsistent execution_version: {versions}"
    print("  ✓ Execution version is 'v5_scheduler'")
    
    # Check 4: Exactly one 'total' row per wheel_id
    totals = exits_df[exits_df['phase'] == 'total']
    if len(totals) > 0:
        assert totals['wheel_id'].is_unique, "Duplicate 'total' rows detected for same wheel_id"
        print("  ✓ Exactly one 'total' row per wheel_id")
    else:
        print("  ⚠ No 'total' phase rows found")
    
    # Check 5: CSP exits have required fields
    csp_exits = exits_df[exits_df['phase'] == 'csp']
    if len(csp_exits) > 0:
        csp_required = ['symbol', 'entry_date', 'exit_reason']
        csp_missing = [col for col in csp_required if col not in csp_exits.columns]
        assert len(csp_missing) == 0, f"CSP exits missing fields: {csp_missing}"
        print(f"  ✓ CSP exits have required fields: {csp_required}")
    
    print("=" * 60)
    print("ALL VALIDATIONS PASSED")
    print("=" * 60)
    
    return True


print("=" * 60)
print("V5 VALIDATION FUNCTIONS LOADED")
print("=" * 60)
print("  - validate_v5_results(exits_df)")
print("=" * 60)


V5 VALIDATION FUNCTIONS LOADED
  - validate_v5_results(exits_df)


## V5 Scheduler Execution

Run the scheduler over the configured date range and symbols.


In [44]:
# =============================================================================
# RUN V5 SCHEDULER
# =============================================================================

print("=" * 60)
print("RUNNING V5 SCHEDULER BACKTEST")
print("=" * 60)

# Run the scheduler
v5_results_df = run_v5_scheduler(CONFIG, CC_CONFIG, SCHEDULER_CONFIG, client)

# Validate results
if len(v5_results_df) > 0:
    validate_v5_results(v5_results_df)
    
    # Aggregate and print summary
    v5_stats = aggregate_v5_results(v5_results_df)
    print_v5_summary(v5_results_df, v5_stats)
else:
    print("⚠ No results to analyze")


RUNNING V5 SCHEDULER BACKTEST
V5 SCHEDULER STARTING
Date range: 2023-06-06 to 2023-09-13
Trading days: 69
Symbols: ['TSLA']
[2023-06-06] TSLA: 7 candidates
    [CACHE HIT] Loading daily prices for TSLA  230707P00205000
    [CACHE HIT] Loading daily prices for TSLA  230707P00210000
    [CACHE HIT] Loading daily prices for TSLA  230714P00200000
    [CACHE HIT] Loading daily prices for TSLA  230714P00205000
    [CACHE HIT] Loading daily prices for TSLA  230714P00210000
    [CACHE HIT] Loading daily prices for TSLA  230721P00200000
    [CACHE HIT] Loading daily prices for TSLA  230721P00205000
[2023-06-07] TSLA: 5 candidates
    [CACHE HIT] Loading daily prices for TSLA  230707P00210000
    [CACHE HIT] Loading daily prices for TSLA  230714P00205000
    [CACHE HIT] Loading daily prices for TSLA  230714P00210000
    [CACHE HIT] Loading daily prices for TSLA  230721P00205000
    [CACHE HIT] Loading daily prices for TSLA  230721P00210000
[2023-06-08] TSLA: 4 candidates
    [CACHE HIT] Loading 

/var/folders/6k/0v57cgbd2k37vp0lh44zby640000gn/T/ipykernel_14665/644160136.py:41: BentoWarning: No data found for the request you submitted.
  equity_data = client.timeseries.get_range(


    [CACHE HIT] Loading daily prices for TSLA  230818P00260000


/var/folders/6k/0v57cgbd2k37vp0lh44zby640000gn/T/ipykernel_14665/644160136.py:41: BentoWarning: No data found for the request you submitted.
  equity_data = client.timeseries.get_range(


    [CACHE HIT] Loading daily prices for TSLA  230818P00265000


...

    [CACHE HIT] Loading daily prices for TSLA  230901P00245000
    [CACHE HIT] Loading daily prices for TSLA  230901P00250000
[2023-07-27] TSLA: 5 candidates
    [CACHE HIT] Loading daily prices for TSLA  230901P00240000
    [CACHE HIT] Loading daily prices for TSLA  230901P00245000
    [CACHE HIT] Loading daily prices for TSLA  230908P00235000
    [CACHE HIT] Loading daily prices for TSLA  230908P00240000
    [CACHE HIT] Loading daily prices for TSLA  230908P00245000
[2023-07-28] TSLA: 4 candidates
    [CACHE HIT] Loading daily prices for TSLA  230901P00250000
    [CACHE HIT] Loading daily prices for TSLA  230901P00255000
    [CACHE HIT] Loading daily prices for TSLA  230908P00245000
    [CACHE HIT] Loading daily prices for TSLA  230908P00250000
[2023-07-31] TSLA: 4 candidates
    [CACHE HIT] Loading daily prices for TSLA  230901P00250000
    [CACHE HIT] Loading daily prices for TSLA  230901P00255000
    [CACHE HIT] Loading daily prices for TSLA  230908P00250000
    [CACHE HIT] Loadin

/var/folders/6k/0v57cgbd2k37vp0lh44zby640000gn/T/ipykernel_14665/644160136.py:41: BentoWarning: No data found for the request you submitted.
  equity_data = client.timeseries.get_range(


[2023-09-12] TSLA: 7 candidates
    [CACHE HIT] Loading daily prices for TSLA  231013P00250000
    [CACHE HIT] Loading daily prices for TSLA  231013P00255000
    [CACHE HIT] Loading daily prices for TSLA  231020P00245000
    [CACHE HIT] Loading daily prices for TSLA  231020P00250000
    [CACHE HIT] Loading daily prices for TSLA  231020P00255000
    [CACHE HIT] Loading daily prices for TSLA  231027P00245000
    [CACHE HIT] Loading daily prices for TSLA  231027P00250000
[2023-09-13] TSLA: 7 candidates
    [CACHE HIT] Loading daily prices for TSLA  231013P00250000
    [CACHE HIT] Loading daily prices for TSLA  231013P00255000
    [CACHE HIT] Loading daily prices for TSLA  231020P00250000
    [CACHE HIT] Loading daily prices for TSLA  231020P00255000
    [CACHE HIT] Loading daily prices for TSLA  231027P00245000
    [CACHE HIT] Loading daily prices for TSLA  231027P00250000
    [CACHE HIT] Loading daily prices for TSLA  231027P00255000


/var/folders/6k/0v57cgbd2k37vp0lh44zby640000gn/T/ipykernel_14665/644160136.py:41: BentoWarning: No data found for the request you submitted.
  equity_data = client.timeseries.get_range(



SCHEDULER COMPLETE
Wheels launched: 377
Total result records: 754
V5 DATA MODEL VALIDATION
  ✓ Every wheel_id has at least one phase
  ✓ Required fields present: ['wheel_id', 'phase', 'execution_version']
  ✓ Execution version is 'v5_scheduler'
  ✓ Exactly one 'total' row per wheel_id
  ✓ CSP exits have required fields: ['symbol', 'entry_date', 'exit_reason']
ALL VALIDATIONS PASSED

V5 SCHEDULER RESULTS SUMMARY

────────────────────────────────────────
AGGREGATE STATISTICS
────────────────────────────────────────
Total Wheels:      377
Total P&L:         $190,932.18
Avg Wheel ROC:     2.09%
Median Wheel ROC:  1.88%
Win Rate:          100.0%
Avg Days Held:     15.6
Max Loss (proxy):  $58.58

────────────────────────────────────────
PHASE BREAKDOWN
────────────────────────────────────────
phase
csp      377
total    377

────────────────────────────────────────
CSP EXIT REASONS
────────────────────────────────────────
exit_reason
profit_target        336
expired_worthless     41



In [59]:
# =============================================================================
# V5 RESULTS EXPLORATION
# =============================================================================

if len(v5_results_df) > 0:
    print("=" * 60)
    print("V5 RESULTS DATA")
    print("=" * 60)
    print(f"\nTotal records: {len(v5_results_df)}")
    print(f"Unique wheels: {v5_results_df['wheel_id'].nunique()}")
    
    # Show phase breakdown
    print(f"\nPhase breakdown:")
    print(v5_results_df['phase'].value_counts())
    
    # Display sample of wheel summaries
    totals = v5_results_df[v5_results_df['phase'] == 'total']
    if len(totals) > 0:
        display_cols = ['wheel_id', 'state', 'total_pnl', 'wheel_roc', 'total_days', 
                       'csp_exit_reason', 'cc_exit_reason']
        available_cols = [c for c in display_cols if c in totals.columns]
        print(f"\nSample wheel summaries (first 10):")
        display(totals[available_cols].head(10).round(2))
    
    # Display sample CSP exits
    csp_exits = v5_results_df[v5_results_df['phase'] == 'csp']
    if len(csp_exits) > 0:
        csp_cols = ['wheel_id', 'symbol', 'entry_date', 'exit_date', 'exit_reason', 
                   'premium', 'exit_price', 'exit_pnl', 'roc']
        available_csp_cols = [c for c in csp_cols if c in csp_exits.columns]
        print(f"\nSample CSP exits (first 10):")
        display(csp_exits[available_csp_cols].head(10).round(2))


V5 RESULTS DATA

Total records: 754
Unique wheels: 377

Phase breakdown:
phase
csp      377
total    377
Name: count, dtype: int64

Sample wheel summaries (first 10):


,wheel_id,state,total_pnl,wheel_roc,total_days,csp_exit_reason,cc_exit_reason
1,TSLA_20230606_0,WHEEL_COMPLETE,411.28,2.01,2.0,profit_target,NaN
3,TSLA_20230606_1,WHEEL_COMPLETE,484.68,2.31,2.0,profit_target,NaN
5,TSLA_20230606_2,WHEEL_COMPLETE,347.38,1.74,2.0,profit_target,NaN
7,TSLA_20230606_3,WHEEL_COMPLETE,442.68,2.16,2.0,profit_target,NaN
9,TSLA_20230606_4,WHEEL_COMPLETE,553.58,2.64,2.0,profit_target,NaN
11,TSLA_20230606_5,WHEEL_COMPLETE,388.68,1.94,2.0,profit_target,NaN
13,TSLA_20230606_6,WHEEL_COMPLETE,467.68,2.28,2.0,profit_target,NaN
15,TSLA_20230607_7,WHEEL_COMPLETE,343.68,1.64,1.0,profit_target,NaN
17,TSLA_20230607_8,WHEEL_COMPLETE,289.50,1.41,1.0,profit_target,NaN
19,TSLA_20230607_9,WHEEL_COMPLETE,374.68,1.78,1.0,profit_target,NaN



Sample CSP exits (first 10):


,wheel_id,symbol,entry_date,exit_date,exit_reason,premium,exit_price,exit_pnl,roc
0,TSLA_20230606_0,TSLA 230707P00205000,2023-06-06,2023-06-08 20:00:00,profit_target,634.00,221.40,411.28,2.01
2,TSLA_20230606_1,TSLA 230707P00210000,2023-06-06,2023-06-08 20:00:00,profit_target,806.00,320.00,484.68,2.31
4,TSLA_20230606_2,TSLA 230714P00200000,2023-06-06,2023-06-08 20:00:00,profit_target,599.00,250.30,347.38,1.74
6,TSLA_20230606_3,TSLA 230714P00205000,2023-06-06,2023-06-08 20:00:00,profit_target,749.00,305.00,442.68,2.16
8,TSLA_20230606_4,TSLA 230714P00210000,2023-06-06,2023-06-08 20:00:00,profit_target,918.00,363.10,553.58,2.64
10,TSLA_20230606_5,TSLA 230721P00200000,2023-06-06,2023-06-08 20:00:00,profit_target,782.00,392.00,388.68,1.94
12,TSLA_20230606_6,TSLA 230721P00205000,2023-06-06,2023-06-08 20:00:00,profit_target,951.00,482.00,467.68,2.28
14,TSLA_20230607_7,TSLA 230707P00210000,2023-06-07,2023-06-08 20:00:00,profit_target,665.00,320.00,343.68,1.64
16,TSLA_20230607_8,TSLA 230714P00205000,2023-06-07,2023-06-08 20:00:00,profit_target,614.65,323.83,289.50,1.41
18,TSLA_20230607_9,TSLA 230714P00210000,2023-06-07,2023-06-08 20:00:00,profit_target,776.00,400.00,374.68,1.78


In [60]:
# =============================================================================
# V5 ANALYSIS BY DATE
# =============================================================================

if len(v5_results_df) > 0 and 'scheduler_entry_date' in v5_results_df.columns:
    print("=" * 60)
    print("ANALYSIS BY ENTRY DATE")
    print("=" * 60)
    
    # Get wheel summaries
    totals = v5_results_df[v5_results_df['phase'] == 'total'].copy()
    
    if len(totals) > 0:
        # Group by entry date
        totals['entry_date_str'] = totals['scheduler_entry_date']
        daily_summary = totals.groupby('entry_date_str').agg({
            'wheel_id': 'count',
            'total_pnl': 'sum',
            'wheel_roc': 'mean',
        }).rename(columns={
            'wheel_id': 'wheels',
            'total_pnl': 'total_pnl',
            'wheel_roc': 'avg_roc'
        })
        
        print(f"\nDaily Summary (first 20 days with trades):")
        print(daily_summary.head(20).round(2))
        
        # Monthly summary if enough data
        totals['entry_month'] = pd.to_datetime(totals['scheduler_entry_date']).dt.to_period('M')
        monthly_summary = totals.groupby('entry_month').agg({
            'wheel_id': 'count',
            'total_pnl': 'sum',
            'wheel_roc': 'mean',
        }).rename(columns={
            'wheel_id': 'wheels',
            'total_pnl': 'total_pnl',
            'wheel_roc': 'avg_roc'
        })
        
        print(f"\nMonthly Summary:")
        print(monthly_summary.round(2))


ANALYSIS BY ENTRY DATE

Daily Summary (first 20 days with trades):
                wheels  total_pnl  avg_roc
entry_date_str                            
2023-06-06           7    3095.96     2.15
2023-06-07           5    1756.72     1.69
2023-06-08           4    1827.82     2.10
2023-06-09           5    2359.20     2.09
2023-06-12           5    2238.60     1.93
2023-06-13           8    4704.24     2.45
2023-06-14           8    4587.14     2.43
2023-06-15           6    3762.18     2.66
2023-06-16           6    4520.18     3.13
2023-06-20           5    2716.95     2.16
2023-06-21           9    6703.62     3.09
2023-06-22           6    3977.44     2.70
2023-06-23           6    4070.60     2.88
2023-06-26           5    2399.39     2.14
2023-06-27           5    3161.00     2.73
2023-06-28           4    2234.32     2.35
2023-06-29           4    2118.08     2.23
2023-06-30           4    1884.22     1.94
2023-07-05           8    3459.24     1.65
2023-07-06           5    2446

## Hedge Fund Performance Metrics & Dashboard

Calculate institutional-grade performance metrics and visualize results.

In [65]:
# =============================================================================
# HEDGE FUND PERFORMANCE METRICS
# =============================================================================

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime

def calculate_hf_metrics(results_df, initial_capital=100000, risk_free_rate=0.05):
    """
    Calculate comprehensive hedge fund performance metrics.
    
    Args:
        results_df: DataFrame from run_v5_scheduler()
        initial_capital: Starting capital for the strategy
        risk_free_rate: Annual risk-free rate (default 5%)
    
    Returns:
        dict: Comprehensive performance metrics
    """
    if len(results_df) == 0:
        return None
    
    # Get wheel-level results (phase == 'total')
    totals = results_df[results_df['phase'] == 'total'].copy()
    
    if len(totals) == 0:
        print("⚠ No 'total' phase rows found")
        return None
    
    # Determine P&L column
    pnl_col = 'total_pnl' if 'total_pnl' in totals.columns else 'exit_pnl'
    
    # Sort by entry date
    totals['entry_date_parsed'] = pd.to_datetime(totals['scheduler_entry_date'])
    totals = totals.sort_values('entry_date_parsed')
    
    # =========================================================================
    # BASIC STATISTICS
    # =========================================================================
    total_pnl = totals[pnl_col].sum()
    num_trades = len(totals)
    winners = totals[totals[pnl_col] > 0]
    losers = totals[totals[pnl_col] < 0]
    num_winners = len(winners)
    num_losers = len(losers)
    
    # Win/Loss metrics
    win_rate = (num_winners / num_trades * 100) if num_trades > 0 else 0
    avg_win = winners[pnl_col].mean() if num_winners > 0 else 0
    avg_loss = abs(losers[pnl_col].mean()) if num_losers > 0 else 0
    largest_win = winners[pnl_col].max() if num_winners > 0 else 0
    largest_loss = losers[pnl_col].min() if num_losers > 0 else 0
    
    # Profit Factor = Gross Profits / Gross Losses
    gross_profit = winners[pnl_col].sum() if num_winners > 0 else 0
    gross_loss = abs(losers[pnl_col].sum()) if num_losers > 0 else 1  # Avoid div by 0
    profit_factor = gross_profit / gross_loss if gross_loss > 0 else float('inf')
    
    # Expectancy = (Win Rate * Avg Win) - (Loss Rate * Avg Loss)
    loss_rate = num_losers / num_trades if num_trades > 0 else 0
    expectancy = (win_rate/100 * avg_win) - (loss_rate * avg_loss)
    
    # =========================================================================
    # CAPITAL-ADJUSTED METRICS (FIX: use actual capital deployed)
    # =========================================================================
    # Total capital deployed = sum of all initial_capital per trade
    total_capital_deployed = totals['initial_capital'].sum() if 'initial_capital' in totals.columns else initial_capital * num_trades
    
    # =========================================================================
    # EQUITY CURVE & DRAWDOWN
    # =========================================================================
    # Build cumulative equity curve
    totals['cumulative_pnl'] = totals[pnl_col].cumsum()
    totals['equity'] = initial_capital + totals['cumulative_pnl']
    
    # Calculate drawdown
    totals['peak'] = totals['equity'].cummax()
    totals['drawdown'] = totals['equity'] - totals['peak']
    totals['drawdown_pct'] = (totals['drawdown'] / totals['peak']) * 100
    
    max_drawdown = totals['drawdown'].min()
    max_drawdown_pct = totals['drawdown_pct'].min()
    
    # =========================================================================
    # RETURN METRICS (FIXED: use capital deployed)
    # =========================================================================
    # Per-trade ROC is already calculated correctly
    avg_roc = totals['wheel_roc'].mean() if 'wheel_roc' in totals.columns else 0
    
    # Total return based on hypothetical single capital base (for equity curve)
    total_return_curve = (total_pnl / initial_capital) * 100
    
    # Realistic return: P&L / total capital deployed
    total_return_realistic = (total_pnl / total_capital_deployed) * 100 if total_capital_deployed > 0 else 0
    
    final_equity = initial_capital + total_pnl
    
    # Calculate trading period
    start_date = totals['entry_date_parsed'].min()
    end_date = totals['entry_date_parsed'].max()
    trading_days = (end_date - start_date).days
    years = trading_days / 365.25 if trading_days > 0 else 1
    
    # CAGR based on realistic return
    if years > 0 and total_capital_deployed > 0:
        # Annualized ROC (average per-trade ROC * trades per year)
        trades_per_year = num_trades / years
        cagr = avg_roc * trades_per_year  # Approximate annualized return
    else:
        cagr = 0
    
    # =========================================================================
    # RISK-ADJUSTED METRICS (FIXED: use per-trade ROC)
    # =========================================================================
    roc_col = 'wheel_roc' if 'wheel_roc' in totals.columns else 'roc'
    if roc_col in totals.columns:
        returns = totals[roc_col] / 100  # Convert to decimal
    else:
        returns = totals[pnl_col] / total_capital_deployed * num_trades
    
    returns_std = returns.std()
    returns_mean = returns.mean()
    
    # Annualize
    trades_per_year = num_trades / years if years > 0 else num_trades
    annualized_return = returns_mean * trades_per_year
    annualized_vol = returns_std * np.sqrt(trades_per_year) if trades_per_year > 0 else 0
    
    # Sharpe Ratio
    sharpe_ratio = (annualized_return - risk_free_rate) / annualized_vol if annualized_vol > 0 else 0
    
    # Sortino Ratio
    negative_returns = returns[returns < 0]
    downside_std = negative_returns.std() if len(negative_returns) > 0 else 0
    annualized_downside = downside_std * np.sqrt(trades_per_year) if trades_per_year > 0 else 0
    sortino_ratio = (annualized_return - risk_free_rate) / annualized_downside if annualized_downside > 0 else float('inf')
    
    # Calmar Ratio
    calmar_ratio = abs(cagr / max_drawdown_pct) if max_drawdown_pct != 0 else float('inf')
    
    # Recovery Factor
    recovery_factor = abs(total_pnl / max_drawdown) if max_drawdown != 0 else float('inf')
    
    # =========================================================================
    # TIME-BASED METRICS
    # =========================================================================
    avg_days_held = totals['total_days'].mean() if 'total_days' in totals.columns else 0
    
    # Monthly returns
    totals['entry_month'] = totals['entry_date_parsed'].dt.to_period('M')
    monthly_pnl = totals.groupby('entry_month')[pnl_col].sum()
    monthly_returns = monthly_pnl / initial_capital * 100
    
    best_month = monthly_returns.max() if len(monthly_returns) > 0 else 0
    worst_month = monthly_returns.min() if len(monthly_returns) > 0 else 0
    
    # =========================================================================
    # COMPILE RESULTS
    # =========================================================================
    metrics = {
        # Basic Stats
        'total_trades': num_trades,
        'winning_trades': num_winners,
        'losing_trades': num_losers,
        'win_rate': win_rate,
        
        # P&L
        'total_pnl': total_pnl,
        'gross_profit': gross_profit,
        'gross_loss': -abs(gross_loss) if gross_loss != 1 else 0,
        'avg_win': avg_win,
        'avg_loss': -avg_loss,
        'largest_win': largest_win,
        'largest_loss': largest_loss,
        'profit_factor': profit_factor,
        'expectancy': expectancy,
        
        # Capital
        'total_capital_deployed': total_capital_deployed,
        'avg_capital_per_trade': total_capital_deployed / num_trades if num_trades > 0 else 0,
        
        # Returns
        'total_return_pct': total_return_realistic,  # Realistic return
        'total_return_curve_pct': total_return_curve,  # For equity curve display
        'avg_roc_per_trade': avg_roc,
        'cagr': cagr,
        'annualized_return': annualized_return * 100,
        'annualized_volatility': annualized_vol * 100,
        
        # Risk Metrics
        'max_drawdown': max_drawdown,
        'max_drawdown_pct': max_drawdown_pct,
        'sharpe_ratio': sharpe_ratio,
        'sortino_ratio': sortino_ratio if sortino_ratio != float('inf') else 99.99,
        'calmar_ratio': calmar_ratio if calmar_ratio != float('inf') else 99.99,
        'recovery_factor': recovery_factor if recovery_factor != float('inf') else 99.99,
        
        # Time
        'start_date': start_date,
        'end_date': end_date,
        'trading_days': trading_days,
        'avg_days_held': avg_days_held,
        'best_month_pct': best_month,
        'worst_month_pct': worst_month,
        
        # Data for plotting
        '_equity_curve': totals[['entry_date_parsed', 'equity', 'drawdown_pct']].copy(),
        '_monthly_returns': monthly_returns,
        '_trade_pnl': totals[pnl_col].values,
    }
    
    return metrics


def print_hf_metrics(metrics):
    """Print formatted hedge fund metrics."""
    if metrics is None:
        print("⚠ No metrics to display")
        return
    
    print("\n" + "=" * 70)
    print("  HEDGE FUND PERFORMANCE REPORT")
    print("=" * 70)
    
    print(f"\n{'─'*35} OVERVIEW {'─'*34}")
    print(f"  Period:              {metrics['start_date'].strftime('%Y-%m-%d')} to {metrics['end_date'].strftime('%Y-%m-%d')} ({metrics['trading_days']} days)")
    print(f"  Total Trades:        {metrics['total_trades']}")
    print(f"  Avg Holding Period:  {metrics['avg_days_held']:.1f} days")
    
    print(f"\n{'─'*35} CAPITAL {'─'*35}")
    print(f"  Total Deployed:      ${metrics['total_capital_deployed']:>12,.0f}")
    print(f"  Avg Per Trade:       ${metrics['avg_capital_per_trade']:>12,.0f}")
    
    print(f"\n{'─'*35} RETURNS {'─'*35}")
    print(f"  Total P&L:           ${metrics['total_pnl']:>12,.2f}")
    print(f"  Return on Capital:   {metrics['total_return_pct']:>12.2f}%  (P&L / Capital Deployed)")
    print(f"  Avg ROC per Trade:   {metrics['avg_roc_per_trade']:>12.2f}%")
    print(f"  Annualized Return:   {metrics['annualized_return']:>12.2f}%  (Avg ROC × Trades/Year)")
    print(f"  Annualized Vol:      {metrics['annualized_volatility']:>12.2f}%")
    
    print(f"\n{'─'*35} RISK METRICS {'─'*30}")
    print(f"  Max Drawdown:        ${metrics['max_drawdown']:>12,.2f} ({metrics['max_drawdown_pct']:.2f}%)")
    print(f"  Sharpe Ratio:        {metrics['sharpe_ratio']:>12.2f}")
    print(f"  Sortino Ratio:       {metrics['sortino_ratio']:>12.2f}")
    print(f"  Calmar Ratio:        {metrics['calmar_ratio']:>12.2f}")
    print(f"  Recovery Factor:     {metrics['recovery_factor']:>12.2f}")
    
    print(f"\n{'─'*35} WIN/LOSS ANALYSIS {'─'*25}")
    print(f"  Win Rate:            {metrics['win_rate']:>12.1f}%")
    print(f"  Winners / Losers:    {metrics['winning_trades']:>5} / {metrics['losing_trades']}")
    print(f"  Profit Factor:       {metrics['profit_factor']:>12.2f}")
    print(f"  Expectancy:          ${metrics['expectancy']:>12.2f}")
    print(f"  Avg Win:             ${metrics['avg_win']:>12.2f}")
    print(f"  Avg Loss:            ${metrics['avg_loss']:>12.2f}")
    print(f"  Largest Win:         ${metrics['largest_win']:>12.2f}")
    print(f"  Largest Loss:        ${metrics['largest_loss']:>12.2f}")
    
    print(f"\n{'─'*35} MONTHLY {'─'*35}")
    print(f"  Best Month:          {metrics['best_month_pct']:>12.2f}%")
    print(f"  Worst Month:         {metrics['worst_month_pct']:>12.2f}%")
    
    print("\n" + "=" * 70)


print("✓ HF Metrics Functions Loaded")
print("  - calculate_hf_metrics(results_df, initial_capital, risk_free_rate)")
print("  - print_hf_metrics(metrics)")

✓ HF Metrics Functions Loaded
  - calculate_hf_metrics(results_df, initial_capital, risk_free_rate)
  - print_hf_metrics(metrics)


In [66]:
# =============================================================================
# PERFORMANCE DASHBOARD
# =============================================================================

def plot_performance_dashboard(metrics, figsize=(16, 12)):
    """Create a comprehensive performance dashboard."""
    if metrics is None:
        print("⚠ No metrics to plot")
        return
    
    plt.style.use('seaborn-v0_8-whitegrid')
    fig = plt.figure(figsize=figsize)
    gs = fig.add_gridspec(3, 3, hspace=0.35, wspace=0.3)
    
    green, red, blue, orange = '#2ecc71', '#e74c3c', '#3498db', '#f39c12'
    
    # 1. EQUITY CURVE
    ax1 = fig.add_subplot(gs[0, :2])
    equity_data = metrics['_equity_curve']
    ax1.plot(equity_data['entry_date_parsed'], equity_data['equity'], color=blue, linewidth=2)
    ax1.axhline(y=100000, color='gray', linestyle='--', alpha=0.5)
    ax1.fill_between(equity_data['entry_date_parsed'], 100000, equity_data['equity'],
                     where=equity_data['equity'] >= 100000, alpha=0.3, color=green)
    ax1.fill_between(equity_data['entry_date_parsed'], 100000, equity_data['equity'],
                     where=equity_data['equity'] < 100000, alpha=0.3, color=red)
    ax1.set_title('Equity Curve', fontsize=14, fontweight='bold')
    ax1.set_ylabel('Portfolio Value ($)')
    ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))
    ax1.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x:,.0f}'))
    
    # 2. KEY METRICS CARD
    ax2 = fig.add_subplot(gs[0, 2])
    ax2.axis('off')
    metrics_text = f"""
┌─────────────────────────────────┐
│      KEY METRICS                │
├─────────────────────────────────┤
│  Capital Deployed: ${metrics['total_capital_deployed']/1e6:.1f}M     │
│  Return on Cap:    {metrics['total_return_pct']:>6.2f}%      │
│  Avg ROC/Trade:    {metrics['avg_roc_per_trade']:>6.2f}%      │
│  Sharpe:           {metrics['sharpe_ratio']:>6.2f}       │
│  Max DD:           {metrics['max_drawdown_pct']:>6.1f}%      │
├─────────────────────────────────┤
│  Win Rate:         {metrics['win_rate']:>6.1f}%      │
│  Profit Factor:    {metrics['profit_factor']:>6.2f}       │
│  Trades:           {metrics['total_trades']:>6d}       │
└─────────────────────────────────┘
"""
    ax2.text(0.5, 0.5, metrics_text, transform=ax2.transAxes, fontsize=10, fontfamily='monospace',
             verticalalignment='center', horizontalalignment='center',
             bbox=dict(boxstyle='round', facecolor='#ecf0f1', edgecolor='#bdc3c7'))
    
    # 3. DRAWDOWN CHART
    ax3 = fig.add_subplot(gs[1, :2])
    ax3.fill_between(equity_data['entry_date_parsed'], 0, equity_data['drawdown_pct'], color=red, alpha=0.5)
    ax3.axhline(y=metrics['max_drawdown_pct'], color='darkred', linestyle='--', alpha=0.7,
                label=f"Max DD: {metrics['max_drawdown_pct']:.1f}%")
    ax3.set_title('Drawdown', fontsize=14, fontweight='bold')
    ax3.set_ylabel('Drawdown (%)')
    ax3.legend(loc='lower left')
    ax3.xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))
    
    # 4. TRADE P&L DISTRIBUTION
    ax4 = fig.add_subplot(gs[1, 2])
    trade_pnl = metrics['_trade_pnl']
    ax4.hist(trade_pnl, bins=30, color=blue, alpha=0.7, edgecolor='white')
    ax4.axvline(x=0, color='gray', linestyle='--', alpha=0.7)
    ax4.axvline(x=np.mean(trade_pnl), color=green, linewidth=2, label=f"Mean: ${np.mean(trade_pnl):,.0f}")
    ax4.set_title('Trade P&L Distribution', fontsize=14, fontweight='bold')
    ax4.set_xlabel('P&L ($)')
    ax4.legend(fontsize=9)
    
    # 5. MONTHLY RETURNS
    ax5 = fig.add_subplot(gs[2, :2])
    monthly_returns = metrics['_monthly_returns']
    months = [str(m) for m in monthly_returns.index]
    values = monthly_returns.values
    colors_bar = [green if v >= 0 else red for v in values]
    bars = ax5.bar(range(len(months)), values, color=colors_bar, alpha=0.8)
    ax5.axhline(y=0, color='gray', linestyle='-', alpha=0.5)
    ax5.set_title('Monthly Returns (%)', fontsize=14, fontweight='bold')
    ax5.set_xticks(range(len(months)))
    ax5.set_xticklabels(months, rotation=45, ha='right')
    
    # 6. WIN/LOSS PIE
    ax6 = fig.add_subplot(gs[2, 2])
    sizes = [metrics['winning_trades'], metrics['losing_trades']]
    if sum(sizes) > 0:
        ax6.pie(sizes, labels=[f"Win\n{sizes[0]}", f"Loss\n{sizes[1]}"], colors=[green, red],
                autopct='%1.1f%%', startangle=90)
    ax6.set_title('Win/Loss', fontsize=14, fontweight='bold')
    
    fig.suptitle(f"Wheel Strategy Dashboard: {metrics['start_date'].strftime('%Y-%m-%d')} to {metrics['end_date'].strftime('%Y-%m-%d')}",
                 fontsize=14, fontweight='bold', y=1.01)
    plt.tight_layout()
    plt.show()
    return fig

print("✓ Dashboard Functions Loaded")

✓ Dashboard Functions Loaded


# =============================================================================
# RUN HF METRICS & DASHBOARD
# =============================================================================

if 'v5_results_df' in dir() and len(v5_results_df) > 0:
    # Calculate metrics
    hf_metrics = calculate_hf_metrics(
        v5_results_df,
        initial_capital=100000,
        risk_free_rate=0.05
    )
    
    # Print report
    print_hf_metrics(hf_metrics)
    
    # Plot dashboard
    plot_performance_dashboard(hf_metrics)
else:
    print("⚠ v5_results_df not found. Run the scheduler first (cells 20-22).")

In [67]:
# =============================================================================
# METRICS SUMMARY TABLE (for comparison across runs)
# =============================================================================

def metrics_to_dataframe(metrics, run_name='Current'):
    """Convert metrics dict to a DataFrame row for comparison."""
    if metrics is None:
        return pd.DataFrame()
    
    # Safely get values with defaults for backwards compatibility
    summary = {
        'Run': run_name,
        'Trades': metrics.get('total_trades', 0),
        'Total P&L': f"${metrics.get('total_pnl', 0):,.0f}",
        'Capital Deployed': f"${metrics.get('total_capital_deployed', 0):,.0f}",
        'ROC': f"{metrics.get('total_return_pct', 0):.2f}%",
        'Avg ROC/Trade': f"{metrics.get('avg_roc_per_trade', 0):.2f}%",
        'Win Rate': f"{metrics.get('win_rate', 0):.1f}%",
        'Sharpe': f"{metrics.get('sharpe_ratio', 0):.2f}",
        'Max DD': f"{metrics.get('max_drawdown_pct', 0):.1f}%",
        'Profit Factor': f"{metrics.get('profit_factor', 0):.2f}",
    }
    return pd.DataFrame([summary])

# Display summary table
if 'hf_metrics' in dir() and hf_metrics is not None:
    print("=" * 70)
    print("METRICS SUMMARY (for comparison)")
    print("=" * 70)
    summary_df = metrics_to_dataframe(hf_metrics, run_name='V5 Scheduler')
    display(summary_df.T)

METRICS SUMMARY (for comparison)


,0
Run,V5 Scheduler
Trades,377
Total P&L,"$190,932"
Capital Deployed,$0
ROC,190.93%
Avg ROC/Trade,0.00%
Win Rate,100.0%
Sharpe,77.28
Max DD,0.0%
Profit Factor,190932.18
